In [1]:
!pip install yfinance


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import yfinance as yf
import pandas as pd

# List of tickers to analyze
tickers = ['^VIX', '^TNX', '^IRX',
           'VXX', 'UVXY',
           'MTUM', 
           'SPY', 'QQQ', 'QQQM', 'VOOG', 'VGT', 'XLU',
           'GLD', 'IAU', 'PHYS',
           'TLT', 'TMF', 'SHY', 'EDV', 'ZROZ', 'GOVT',
           'AAPL', 'SFM', 'PLTR', 'QUBT', 'IONQ', 'APLD', 'LMND', 'VRT', 'RKLB', 'AFRM', 'FLR', 'ESOA', 'BKNG', 'HQY', 'NU', 'VNO', 'MP', 'SMR', 'IDCC', 'RVMD', 'DHI', 'PANW', 'DASH', 'CHWY', 'WPM',
           'CL', 'LMT', 'TKO', 'T', 'SE', 'GDX', 'COST', 'GOOG', 'SLV', 'NEE', 'CCJ', 'IAU', 'LNTH', 'GEVO']

# Initialize a dictionary to store results for each ticker
results = {}

# Loop through each ticker
for ticker in tickers:
    # Download data for each ticker
    data = yf.download(ticker, start="2022-01-01", end="2024-12-31", interval="1d")

    # Calculate moving averages
    data['3_SMA'] = data['Close'].rolling(window=3).mean()
    data['5_SMA'] = data['Close'].rolling(window=5).mean()
    data['20_SMA'] = data['Close'].rolling(window=20).mean()
    data['50_SMA'] = data['Close'].rolling(window=50).mean()

    # Determine Market Trend
    data['Market_Trend'] = 0
    data.loc[data['3_SMA'] > data['5_SMA'], 'Market_Trend'] = 1
    data.loc[data['3_SMA'] < data['5_SMA'], 'Market_Trend'] = -1

    # Calculate the Price Change %
    data['Price_Change%'] = data['Close'].pct_change() * 100 

    # Cumulative returns for Maximum Drawdown Calculation
    data['Cumulative_Returns'] = (1 + data['Price_Change%'] / 100).cumprod()

    # Calculate Maximum Drawdown
    data['Previous_Peak'] = data['Cumulative_Returns'].cummax()
    data['Drawdown'] = (data['Cumulative_Returns'] - data['Previous_Peak']) / data['Previous_Peak']
    max_drawdown = data['Drawdown'].min()

    # Initialize columns for cumulative change calculation
    data['Cumulative_Change%'] = 0.0
    current_base_price = None
    current_trend = None
    previous_close = None

    for index in data.index:
        market_trend = data.at[index, 'Market_Trend']
        close_price = data.at[index, 'Close']

        if market_trend != current_trend:
            if previous_close is not None:
                data.at[index, 'Cumulative_Change%'] = ((close_price - previous_close) / previous_close) * 100
            
            current_base_price = previous_close
        else:
            if current_base_price is not None:
                data.at[index, 'Cumulative_Change%'] = ((close_price - current_base_price) / current_base_price) * 100
        
        previous_close = close_price
        current_trend = market_trend

    trend_indicator = data[['Close', 'Price_Change%', 'Market_Trend', 'Cumulative_Change%']].dropna()
    trend_indicator.columns = [f'{ticker} Close',
                               f'{ticker} daily move%',
                               f'{ticker} trend signal',
                               f'{ticker} cumulative change since signal']
    
    # Adding Maximum Drawdown to the results
    trend_indicator[f'{ticker} Max Drawdown'] = max_drawdown

    results[ticker] = trend_indicator

# Function to get the current risk-free rate from the results dictionary
def get_risk_free_rate():
    treasury_yield = results["^TNX"].iloc[-1]['^TNX Close']
    return treasury_yield / 100

# Get the risk-free rate converted from annual to daily
risk_free_rate = get_risk_free_rate() / 252 

# Calculate and append Sharpe Ratio to each trend_indicator DataFrame
for ticker in tickers:
    if ticker in results:
        average_return = results[ticker][f'{ticker} daily move%'].rolling(window=7).mean()
        rolling_std_dev_returns = results[ticker][f'{ticker} daily move%'].rolling(window=7).std()

        sharpe_ratio = (average_return - risk_free_rate) / rolling_std_dev_returns
        results[ticker][f'{ticker} Sharpe Ratio'] = sharpe_ratio

# Display the results with Sharpe Ratio and Max Drawdown for each ticker
for ticker in tickers:
    display(results[ticker].tail(1))


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,^VIX Close,^VIX daily move%,^VIX trend signal,^VIX cumulative change since signal,^VIX Max Drawdown,^VIX Sharpe Ratio
Date,,,,,,
2024-11-06,15.84,-22.693995,-1,-27.934484,-0.674623,-0.22498


,^TNX Close,^TNX daily move%,^TNX trend signal,^TNX cumulative change since signal,^TNX Max Drawdown,^TNX Sharpe Ratio
Date,,,,,,
2024-11-05,4.289,-0.464144,1,5.303218,-0.274058,0.195357


,^IRX Close,^IRX daily move%,^IRX trend signal,^IRX cumulative change since signal,^IRX Max Drawdown,^IRX Sharpe Ratio
Date,,,,,,
2024-11-05,4.423,0.113165,-1,-1.711114,-0.2925,-0.625596


,VXX Close,VXX daily move%,VXX trend signal,VXX cumulative change since signal,VXX Max Drawdown,VXX Sharpe Ratio
Date,,,,,,
2024-11-05,51.549999,-5.360753,-1,-5.360753,-0.911836,-0.194249


...

,NEE Close,NEE daily move%,NEE trend signal,NEE cumulative change since signal,NEE Max Drawdown,NEE Sharpe Ratio
Date,,,,,,
2024-11-05,78.650002,0.357278,-1,-7.427022,-0.458022,-0.231258


,CCJ Close,CCJ daily move%,CCJ trend signal,CCJ cumulative change since signal,CCJ Max Drawdown,CCJ Sharpe Ratio
Date,,,,,,
2024-11-05,51.259998,0.215047,-1,-5.038906,-0.348211,-0.337145


,IAU Close,IAU daily move%,IAU trend signal,IAU cumulative change since signal,IAU Max Drawdown,IAU Sharpe Ratio
Date,,,,,,
2024-11-05,51.810001,0.212767,-1,-0.057868,-0.209338,0.00669


,LNTH Close,LNTH daily move%,LNTH trend signal,LNTH cumulative change since signal,LNTH Max Drawdown,LNTH Sharpe Ratio
Date,,,,,,
2024-11-05,114.959999,-0.424424,1,6.464156,-0.487506,0.219648


,GEVO Close,GEVO daily move%,GEVO trend signal,GEVO cumulative change since signal,GEVO Max Drawdown,GEVO Sharpe Ratio
Date,,,,,,
2024-11-05,2.35,-0.843881,-1,-19.243992,-0.901575,-0.640211


In [ ]:
import yfinance as yf
import pandas as pd

# List of tickers to analyze
tickers = ['^VIX', '^TNX', '^IRX',
           'VXX', 'UVXY',
           'MTUM', 
           'SPY', 'QQQ', 'QQQM', 'VOOG', 'VGT', 'XLU',
           'GLD', 'IAU', 'PHYS',
           'TLT', 'TMF', 'SHY', 'EDV', 'ZROZ', 'GOVT',
           'AAPL', 'SFM']

# Initialize a dictionary to store results for each ticker
results = {}

# Loop through each ticker
for ticker in tickers:
    # Download data for each ticker
    data = yf.download(ticker, start="2022-01-01", end="2024-12-31", interval="1d")

    # Calculate moving averages
    data['3_SMA'] = data['Close'].rolling(window=3).mean()
    data['5_SMA'] = data['Close'].rolling(window=5).mean()
    data['20_SMA'] = data['Close'].rolling(window=20).mean()
    data['50_SMA'] = data['Close'].rolling(window=50).mean()

    # Determine Market Trend
    data['Market_Trend'] = 0
    data.loc[data['3_SMA'] > data['5_SMA'], 'Market_Trend'] = 1
    data.loc[data['3_SMA'] < data['5_SMA'], 'Market_Trend'] = -1

    # Calculate the Price Change %
    data['Price_Change%'] = data['Close'].pct_change() * 100 

    # Cumulative returns for Maximum Drawdown Calculation
    data['Cumulative_Returns'] = (1 + data['Price_Change%'] / 100).cumprod()

    # Rolling Maximum Drawdown Calculation
    # Compute the rolling maximum of the cumulative returns
    data['Rolling_Peak'] = data['Cumulative_Returns'].cummax()  
    data['Rolling_Drawdown'] = (data['Cumulative_Returns'] - data['Rolling_Peak']) / data['Rolling_Peak']
    data['Max_Drawdown'] = data['Rolling_Drawdown'].min()  # Minimum of all drawdowns

    # Initialize columns for cumulative change calculation
    data['Cumulative_Change%'] = 0.0
    current_base_price = None
    current_trend = None
    previous_close = None

    for index in data.index:
        market_trend = data.at[index, 'Market_Trend']
        close_price = data.at[index, 'Close']

        if market_trend != current_trend:
            if previous_close is not None:
                data.at[index, 'Cumulative_Change%'] = ((close_price - previous_close) / previous_close) * 100
            
            current_base_price = previous_close
        else:
            if current_base_price is not None:
                data.at[index, 'Cumulative_Change%'] = ((close_price - current_base_price) / current_base_price) * 100
        
        previous_close = close_price
        current_trend = market_trend

    trend_indicator = data[['Close', 'Price_Change%', 'Market_Trend', 'Cumulative_Change%']].dropna()
    trend_indicator.columns = [f'{ticker} Close',
                               f'{ticker} daily move%',
                               f'{ticker} trend signal',
                               f'{ticker} cumulative change since signal']
    
    # Add rolling Max Drawdown to the results
    trend_indicator[f'{ticker} Rolling Max Drawdown'] = data['Rolling_Drawdown'] #well-timed entry: If the Rolling Maximum Drawdown equals 0 for most of the time, it typically indicates that the cumulative returns of the investment have been consistently increasing without experiencing any downward movements significant enough to constitute a drawdown

    results[ticker] = trend_indicator

# Function to get the current risk-free rate from the results dictionary
def get_risk_free_rate():
    treasury_yield = results["^TNX"].iloc[-1]['^TNX Close']
    return treasury_yield / 100

# Get the risk-free rate converted from annual to daily
risk_free_rate = get_risk_free_rate() / 252 

# Calculate and append Sharpe Ratio to each trend_indicator DataFrame
for ticker in tickers:
    if ticker in results:
        average_return = results[ticker][f'{ticker} daily move%'].rolling(window=7).mean()
        rolling_std_dev_returns = results[ticker][f'{ticker} daily move%'].rolling(window=7).std()

        sharpe_ratio = (average_return - risk_free_rate) / rolling_std_dev_returns
        results[ticker][f'{ticker} Sharpe Ratio'] = sharpe_ratio

# Display the results with Sharpe Ratio and Rolling Max Drawdown for each ticker
for ticker in tickers:
    display(results[ticker].tail(10))
    


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,^VIX Close,^VIX daily move%,^VIX trend signal,^VIX cumulative change since signal,^VIX Rolling Max Drawdown,^VIX Sharpe Ratio
Date,,,,,,
2024-10-21,18.370001,1.885747,-1,-12.231244,-0.523723,-0.468761
2024-10-22,18.200001,-0.925422,-1,-13.043476,-0.528131,-0.418557
2024-10-23,19.240000,5.714280,1,5.714280,-0.501167,-0.054876
2024-10-24,19.080000,-0.831600,1,4.835160,-0.505315,-0.264946
2024-10-25,20.330000,6.551363,1,11.703292,-0.472906,0.141239
2024-10-28,19.799999,-2.606988,1,8.791200,-0.486648,0.133777
2024-10-29,19.340000,-2.323228,1,6.263733,-0.498574,0.283402
2024-10-30,20.350000,5.222338,1,11.813184,-0.472388,0.378051
2024-10-31,23.160000,13.808351,1,27.252741,-0.399533,0.611832


,^TNX Close,^TNX daily move%,^TNX trend signal,^TNX cumulative change since signal,^TNX Rolling Max Drawdown,^TNX Sharpe Ratio
Date,,,,,,
2024-10-21,4.182,2.676165,1,2.676165,-0.161588,0.201552
2024-10-22,4.204,0.526060,1,3.216304,-0.157177,0.313591
2024-10-23,4.242,0.903904,1,4.149279,-0.149559,0.339625
2024-10-24,4.200,-0.990106,1,3.118091,-0.157979,0.412289
2024-10-25,4.232,0.761906,1,3.903755,-0.151564,0.585109
2024-10-28,4.278,1.086957,1,5.033143,-0.142342,0.527022
2024-10-29,4.274,-0.093495,1,4.934943,-0.143143,0.617507
2024-10-30,4.266,-0.187187,1,4.738518,-0.144747,0.386349
2024-10-31,4.284,0.421944,1,5.180456,-0.141139,0.368572


,^IRX Close,^IRX daily move%,^IRX trend signal,^IRX cumulative change since signal,^IRX Rolling Max Drawdown,^IRX Sharpe Ratio
Date,,,,,,
2024-10-21,4.518,0.110794,1,0.110794,-0.155198,-0.180396
2024-10-22,4.500,-0.398409,1,-0.288057,-0.158564,-0.209640
2024-10-23,4.505,0.111114,-1,0.111114,-0.157629,-0.178067
2024-10-24,4.487,-0.399559,-1,-0.288889,-0.160995,-0.330632
2024-10-25,4.510,0.512597,-1,0.222227,-0.156694,0.067085
2024-10-28,4.482,-0.620850,-1,-0.400003,-0.161930,-0.266642
2024-10-29,4.480,-0.044620,-1,-0.444444,-0.162304,-0.266767
2024-10-30,4.463,-0.379469,-1,-0.822226,-0.165482,-0.447042
2024-10-31,4.432,-0.694592,-1,-1.511108,-0.171279,-0.501327


,VXX Close,VXX daily move%,VXX trend signal,VXX cumulative change since signal,VXX Rolling Max Drawdown,VXX Sharpe Ratio
Date,,,,,,
2024-10-21,51.349998,0.273383,-1,-2.097240,-0.888602,-0.404414
2024-10-22,51.150002,-0.389478,-1,-2.478550,-0.889036,-0.352815
2024-10-23,53.720001,5.024437,1,5.024437,-0.883461,0.168338
2024-10-24,52.810001,-1.693968,1,3.245356,-0.885435,-0.101445
2024-10-25,55.290001,4.696079,1,8.093840,-0.880055,0.181265
2024-10-28,52.709999,-4.666308,1,3.049849,-0.885652,0.034897
2024-10-29,52.480000,-0.436349,1,2.600192,-0.886151,0.116363
2024-10-30,53.759998,2.439022,-1,2.439022,-0.883374,0.201312
2024-10-31,57.869999,7.645091,1,7.645091,-0.874458,0.429387


...

,EDV Close,EDV daily move%,EDV trend signal,EDV cumulative change since signal,EDV Rolling Max Drawdown,EDV Sharpe Ratio
Date,,,,,,
2024-10-21,72.680000,-2.456049,-1,-2.456049,-0.464249,-0.239653
2024-10-22,72.790001,0.151349,-1,-2.308417,-0.463438,-0.176129
2024-10-23,72.580002,-0.288500,-1,-2.590257,-0.464986,-0.186279
2024-10-24,73.230003,0.895566,-1,-1.717889,-0.460195,-0.359084
2024-10-25,72.650002,-0.792028,1,-0.792028,-0.464470,-0.534492
2024-10-28,72.410004,-0.330348,1,-1.119759,-0.466239,-0.379521
2024-10-29,72.559998,0.207145,-1,0.207145,-0.465133,-0.352228
2024-10-30,73.000000,0.606398,-1,0.814799,-0.461890,0.110219
2024-10-31,73.250000,0.342466,1,0.342466,-0.460047,0.154752


,ZROZ Close,ZROZ daily move%,ZROZ trend signal,ZROZ cumulative change since signal,ZROZ Rolling Max Drawdown,ZROZ Sharpe Ratio
Date,,,,,,
2024-10-21,74.589996,-2.953427,-1,-2.953427,-0.494545,-0.233840
2024-10-22,74.980003,0.522868,-1,-2.446002,-0.491902,-0.131496
2024-10-23,74.650002,-0.440120,-1,-2.875357,-0.494138,-0.162906
2024-10-24,75.330002,0.910918,-1,-1.990631,-0.489530,-0.336035
2024-10-25,74.790001,-0.716847,1,-0.716847,-0.493190,-0.474734
2024-10-28,74.580002,-0.280785,1,-0.995619,-0.494613,-0.307494
2024-10-29,74.690002,0.147493,-1,0.147493,-0.493867,-0.319046
2024-10-30,75.349998,0.883647,-1,1.032444,-0.489395,0.225201
2024-10-31,75.570000,0.291972,1,0.291972,-0.487904,0.179087


,GOVT Close,GOVT daily move%,GOVT trend signal,GOVT cumulative change since signal,GOVT Rolling Max Drawdown,GOVT Sharpe Ratio
Date,,,,,,
2024-10-21,22.910000,-0.564241,-1,-0.564241,-0.125906,-0.182166
2024-10-22,22.900000,-0.043650,-1,-0.607644,-0.126288,-0.220693
2024-10-23,22.860001,-0.174668,-1,-0.781251,-0.127814,-0.256271
2024-10-24,22.900000,0.174974,-1,-0.607644,-0.126288,-0.440068
2024-10-25,22.860001,-0.174668,-1,-0.781251,-0.127814,-0.606208
2024-10-28,22.830000,-0.131237,-1,-0.911463,-0.128958,-0.497473
2024-10-29,22.840000,0.043803,-1,-0.868059,-0.128577,-0.536223
2024-10-30,22.820000,-0.087568,-1,-0.954866,-0.129340,-0.440030
2024-10-31,22.820000,0.000000,-1,-0.954866,-0.129340,-0.385996


,AAPL Close,AAPL daily move%,AAPL trend signal,AAPL cumulative change since signal,AAPL Rolling Max Drawdown,AAPL Sharpe Ratio
Date,,,,,,
2024-10-21,236.479996,0.629785,1,3.023439,0.000000,0.478772
2024-10-22,235.860001,-0.262177,1,2.753336,-0.002622,0.575587
2024-10-23,230.759995,-2.162302,1,0.531498,-0.024188,-0.022742
2024-10-24,230.570007,-0.082331,-1,-0.082331,-0.024991,-0.179539
2024-10-25,231.410004,0.364313,-1,0.281682,-0.021439,-0.016980
2024-10-28,233.399994,0.859941,-1,1.144046,-0.013024,0.073400
2024-10-29,233.669998,0.115683,1,0.115683,-0.011883,-0.076951
2024-10-30,230.100006,-1.527792,1,-1.413877,-0.026979,-0.358262
2024-10-31,225.910004,-1.820948,-1,-1.820948,-0.044697,-0.506699


,SFM Close,SFM daily move%,SFM trend signal,SFM cumulative change since signal,SFM Rolling Max Drawdown,SFM Sharpe Ratio
Date,,,,,,
2024-10-21,117.419998,1.398959,-1,1.398959,0.000000,0.426578
2024-10-22,118.269997,0.723896,1,0.723896,0.000000,0.295561
2024-10-23,118.849998,0.490405,1,1.217851,0.000000,0.546363
2024-10-24,118.779999,-0.058898,1,1.158236,-0.000589,0.334875
2024-10-25,119.320000,0.454623,1,1.618124,0.000000,0.422098
2024-10-28,120.180000,0.720751,1,2.350538,0.000000,0.639368
2024-10-29,119.690002,-0.407720,1,1.933235,-0.004077,0.813404
2024-10-30,118.839996,-0.710173,1,1.209332,-0.011150,0.303275
2024-10-31,128.429993,8.069671,1,9.376592,0.000000,0.399102


In [45]:
import yfinance as yf
import pandas as pd

# List of tickers to analyze
tickers = ['^VIX', '^TNX', '^IRX',
           'VXX', 'UVXY',
           'MTUM', 
           'SPY', 'QQQ', 'QQQM', 'VOOG', 'VGT', 'XLU',
           'GLD', 'IAU', 'PHYS',
           'TLT', 'TMF', 'SHY', 'EDV', 'ZROZ', 'GOVT',
           'AAPL', 'SFM']

# Initialize a dictionary to store results for each ticker
results = {}

# Define a target return, for instance, the risk-free rate
target_return = 0.0  # You can set this to your desired target return like the risk-free rate

# Loop through each ticker
for ticker in tickers:
    # Download data for each ticker
    data = yf.download(ticker, start="2022-01-01", end="2024-12-31", interval="1d")

    # Calculate moving averages
    data['3_SMA'] = data['Close'].rolling(window=3).mean()
    data['5_SMA'] = data['Close'].rolling(window=5).mean()
    data['20_SMA'] = data['Close'].rolling(window=20).mean()
    data['50_SMA'] = data['Close'].rolling(window=50).mean()

    # Determine Market Trend
    data['Market_Trend'] = 0
    data.loc[data['3_SMA'] > data['5_SMA'], 'Market_Trend'] = 1
    data.loc[data['3_SMA'] < data['5_SMA'], 'Market_Trend'] = -1

    # Calculate the Price Change %
    data['Price_Change%'] = data['Close'].pct_change() * 100 

    # Cumulative returns for Maximum Drawdown Calculation
    data['Cumulative_Returns'] = (1 + data['Price_Change%'] / 100).cumprod()

    # Rolling Maximum Drawdown Calculation
    data['Rolling_Peak'] = data['Cumulative_Returns'].cummax()
    data['Rolling_Drawdown'] = (data['Cumulative_Returns'] - data['Rolling_Peak']) / data['Rolling_Peak']
    data['Max_Drawdown'] = data['Rolling_Drawdown'].min()

    # Initialization for cumulative change calculation
    data['Cumulative_Change%'] = 0.0
    current_base_price = None
    current_trend = None
    previous_close = None

    for index in data.index:
        market_trend = data.at[index, 'Market_Trend']
        close_price = data.at[index, 'Close']

        if market_trend != current_trend:
            if previous_close is not None:
                data.at[index, 'Cumulative_Change%'] = ((close_price - previous_close) / previous_close) * 100
            
            current_base_price = previous_close
        else:
            if current_base_price is not None:
                data.at[index, 'Cumulative_Change%'] = ((close_price - current_base_price) / current_base_price) * 100
        
        previous_close = close_price
        current_trend = market_trend

    trend_indicator = data[['Close', 'Price_Change%', 'Market_Trend', 'Cumulative_Change%']].dropna()
    trend_indicator.columns = [f'{ticker} Close',
                               f'{ticker} daily move%',
                               f'{ticker} trend signal',
                               f'{ticker} cumulative change since signal']
    
    # Calculate the Average Return and Downside Deviation for Sortino Ratio
    average_return = data['Price_Change%'].mean()  # Average daily return
    downside_returns = data[data['Price_Change%'] < target_return]['Price_Change%']  # Only negative returns
    
    # Calculate downside deviation
    if downside_returns.empty:  # Check if there are no downside returns
        downside_deviation = 0  # Set to zero if no downside returns
    else:
        downside_deviation = (downside_returns ** 2).mean() ** 0.5  # Standard deviation of downside returns

    # Calculate Sortino Ratio
    if downside_deviation != 0:  # Avoid division by zero
        sortino_ratio = (average_return - target_return) / downside_deviation
    else:
        sortino_ratio = float('inf')  # If downside deviation is zero, define it as infinity

    # Extracting the maximum drawdown value
    max_drawdown_value = data['Max_Drawdown'].min()  # Get the minimum value of the max drawdown 

    # Calculate Calmar Ratio
    average_annual_return = ((1 + average_return / 100) ** 252) - 1  # Annualize the average return
    calmar_ratio = average_annual_return / abs(max_drawdown_value) if max_drawdown_value < 0 else float('inf')

    # Add Sortino Ratio and Calmar Ratio to the results
    trend_indicator[f'{ticker} Sortino Ratio'] = sortino_ratio
    trend_indicator[f'{ticker} Calmar Ratio'] = calmar_ratio

    results[ticker] = trend_indicator

# Function to get the current risk-free rate from the results dictionary
def get_risk_free_rate():
    treasury_yield = results["^TNX"].iloc[-1]['^TNX Close']
    return treasury_yield / 100

# Get the risk-free rate converted from annual to daily
risk_free_rate = get_risk_free_rate() / 252 

# Calculate and append Sharpe Ratio to each trend_indicator DataFrame
for ticker in tickers:
    if ticker in results:
        average_return = results[ticker][f'{ticker} daily move%'].rolling(window=7).mean()
        rolling_std_dev_returns = results[ticker][f'{ticker} daily move%'].rolling(window=7).std()

        sharpe_ratio = (average_return - risk_free_rate) / rolling_std_dev_returns
        results[ticker][f'{ticker} Sharpe Ratio'] = sharpe_ratio

# Display the results for each ticker
for ticker in tickers:
    display(results[ticker].tail(10))


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,^VIX Close,^VIX daily move%,^VIX trend signal,^VIX cumulative change since signal,^VIX Sortino Ratio,^VIX Calmar Ratio,^VIX Sharpe Ratio
Date,,,,,,,
2024-10-21,18.370001,1.885747,-1,-12.231244,0.05058,1.39054,-0.468761
2024-10-22,18.200001,-0.925422,-1,-13.043476,0.05058,1.39054,-0.418557
2024-10-23,19.240000,5.714280,1,5.714280,0.05058,1.39054,-0.054876
2024-10-24,19.080000,-0.831600,1,4.835160,0.05058,1.39054,-0.264946
2024-10-25,20.330000,6.551363,1,11.703292,0.05058,1.39054,0.141239
2024-10-28,19.799999,-2.606988,1,8.791200,0.05058,1.39054,0.133777
2024-10-29,19.340000,-2.323228,1,6.263733,0.05058,1.39054,0.283402
2024-10-30,20.350000,5.222338,1,11.813184,0.05058,1.39054,0.378051
2024-10-31,23.160000,13.808351,1,27.252741,0.05058,1.39054,0.611832


,^TNX Close,^TNX daily move%,^TNX trend signal,^TNX cumulative change since signal,^TNX Sortino Ratio,^TNX Calmar Ratio,^TNX Sharpe Ratio
Date,,,,,,,
2024-10-21,4.182,2.676165,1,2.676165,0.079555,1.827391,0.201552
2024-10-22,4.204,0.526060,1,3.216304,0.079555,1.827391,0.313591
2024-10-23,4.242,0.903904,1,4.149279,0.079555,1.827391,0.339625
2024-10-24,4.200,-0.990106,1,3.118091,0.079555,1.827391,0.412289
2024-10-25,4.232,0.761906,1,3.903755,0.079555,1.827391,0.585109
2024-10-28,4.278,1.086957,1,5.033143,0.079555,1.827391,0.527022
2024-10-29,4.274,-0.093495,1,4.934943,0.079555,1.827391,0.617507
2024-10-30,4.266,-0.187187,1,4.738518,0.079555,1.827391,0.386349
2024-10-31,4.284,0.421944,1,5.180456,0.079555,1.827391,0.368572


,^IRX Close,^IRX daily move%,^IRX trend signal,^IRX cumulative change since signal,^IRX Sortino Ratio,^IRX Calmar Ratio,^IRX Sharpe Ratio
Date,,,,,,,
2024-10-21,4.518,0.110794,1,0.110794,0.290397,17.005396,-0.180396
2024-10-22,4.500,-0.398409,1,-0.288057,0.290397,17.005396,-0.209640
2024-10-23,4.505,0.111114,-1,0.111114,0.290397,17.005396,-0.178067
2024-10-24,4.487,-0.399559,-1,-0.288889,0.290397,17.005396,-0.330632
2024-10-25,4.510,0.512597,-1,0.222227,0.290397,17.005396,0.067085
2024-10-28,4.482,-0.620850,-1,-0.400003,0.290397,17.005396,-0.266642
2024-10-29,4.480,-0.044620,-1,-0.444444,0.290397,17.005396,-0.266767
2024-10-30,4.463,-0.379469,-1,-0.822226,0.290397,17.005396,-0.447042
2024-10-31,4.432,-0.694592,-1,-1.511108,0.290397,17.005396,-0.501327


,VXX Close,VXX daily move%,VXX trend signal,VXX cumulative change since signal,VXX Sortino Ratio,VXX Calmar Ratio,VXX Sharpe Ratio
Date,,,,,,,
2024-10-21,51.349998,0.273383,-1,-2.097240,-0.045986,-0.341898,-0.404414
2024-10-22,51.150002,-0.389478,-1,-2.478550,-0.045986,-0.341898,-0.352815
2024-10-23,53.720001,5.024437,1,5.024437,-0.045986,-0.341898,0.168338
2024-10-24,52.810001,-1.693968,1,3.245356,-0.045986,-0.341898,-0.101445
2024-10-25,55.290001,4.696079,1,8.093840,-0.045986,-0.341898,0.181265
2024-10-28,52.709999,-4.666308,1,3.049849,-0.045986,-0.341898,0.034897
2024-10-29,52.480000,-0.436349,1,2.600192,-0.045986,-0.341898,0.116363
2024-10-30,53.759998,2.439022,-1,2.439022,-0.045986,-0.341898,0.201312
2024-10-31,57.869999,7.645091,1,7.645091,-0.045986,-0.341898,0.429387


...

,EDV Close,EDV daily move%,EDV trend signal,EDV cumulative change since signal,EDV Sortino Ratio,EDV Calmar Ratio,EDV Sharpe Ratio
Date,,,,,,,
2024-10-21,72.680000,-2.456049,-1,-2.456049,-0.050155,-0.33161,-0.239653
2024-10-22,72.790001,0.151349,-1,-2.308417,-0.050155,-0.33161,-0.176129
2024-10-23,72.580002,-0.288500,-1,-2.590257,-0.050155,-0.33161,-0.186279
2024-10-24,73.230003,0.895566,-1,-1.717889,-0.050155,-0.33161,-0.359084
2024-10-25,72.650002,-0.792028,1,-0.792028,-0.050155,-0.33161,-0.534492
2024-10-28,72.410004,-0.330348,1,-1.119759,-0.050155,-0.33161,-0.379521
2024-10-29,72.559998,0.207145,-1,0.207145,-0.050155,-0.33161,-0.352228
2024-10-30,73.000000,0.606398,-1,0.814799,-0.050155,-0.33161,0.110219
2024-10-31,73.250000,0.342466,1,0.342466,-0.050155,-0.33161,0.154752


,ZROZ Close,ZROZ daily move%,ZROZ trend signal,ZROZ cumulative change since signal,ZROZ Sortino Ratio,ZROZ Calmar Ratio,ZROZ Sharpe Ratio
Date,,,,,,,
2024-10-21,74.589996,-2.953427,-1,-2.953427,-0.048928,-0.336891,-0.233840
2024-10-22,74.980003,0.522868,-1,-2.446002,-0.048928,-0.336891,-0.131496
2024-10-23,74.650002,-0.440120,-1,-2.875357,-0.048928,-0.336891,-0.162906
2024-10-24,75.330002,0.910918,-1,-1.990631,-0.048928,-0.336891,-0.336035
2024-10-25,74.790001,-0.716847,1,-0.716847,-0.048928,-0.336891,-0.474734
2024-10-28,74.580002,-0.280785,1,-0.995619,-0.048928,-0.336891,-0.307494
2024-10-29,74.690002,0.147493,-1,0.147493,-0.048928,-0.336891,-0.319046
2024-10-30,75.349998,0.883647,-1,1.032444,-0.048928,-0.336891,0.225201
2024-10-31,75.570000,0.291972,1,0.291972,-0.048928,-0.336891,0.179087


,GOVT Close,GOVT daily move%,GOVT trend signal,GOVT cumulative change since signal,GOVT Sortino Ratio,GOVT Calmar Ratio,GOVT Sharpe Ratio
Date,,,,,,,
2024-10-21,22.910000,-0.564241,-1,-0.564241,-0.045062,-0.273292,-0.182166
2024-10-22,22.900000,-0.043650,-1,-0.607644,-0.045062,-0.273292,-0.220693
2024-10-23,22.860001,-0.174668,-1,-0.781251,-0.045062,-0.273292,-0.256271
2024-10-24,22.900000,0.174974,-1,-0.607644,-0.045062,-0.273292,-0.440068
2024-10-25,22.860001,-0.174668,-1,-0.781251,-0.045062,-0.273292,-0.606208
2024-10-28,22.830000,-0.131237,-1,-0.911463,-0.045062,-0.273292,-0.497473
2024-10-29,22.840000,0.043803,-1,-0.868059,-0.045062,-0.273292,-0.536223
2024-10-30,22.820000,-0.087568,-1,-0.954866,-0.045062,-0.273292,-0.440030
2024-10-31,22.820000,0.000000,-1,-0.954866,-0.045062,-0.273292,-0.385996


,AAPL Close,AAPL daily move%,AAPL trend signal,AAPL cumulative change since signal,AAPL Sortino Ratio,AAPL Calmar Ratio,AAPL Sharpe Ratio
Date,,,,,,,
2024-10-21,236.479996,0.629785,1,3.023439,0.025157,0.380939,0.478772
2024-10-22,235.860001,-0.262177,1,2.753336,0.025157,0.380939,0.575587
2024-10-23,230.759995,-2.162302,1,0.531498,0.025157,0.380939,-0.022742
2024-10-24,230.570007,-0.082331,-1,-0.082331,0.025157,0.380939,-0.179539
2024-10-25,231.410004,0.364313,-1,0.281682,0.025157,0.380939,-0.016980
2024-10-28,233.399994,0.859941,-1,1.144046,0.025157,0.380939,0.073400
2024-10-29,233.669998,0.115683,1,0.115683,0.025157,0.380939,-0.076951
2024-10-30,230.100006,-1.527792,1,-1.413877,0.025157,0.380939,-0.358262
2024-10-31,225.910004,-1.820948,-1,-1.820948,0.025157,0.380939,-0.506699


,SFM Close,SFM daily move%,SFM trend signal,SFM cumulative change since signal,SFM Sortino Ratio,SFM Calmar Ratio,SFM Sharpe Ratio
Date,,,,,,,
2024-10-21,117.419998,1.398959,-1,1.398959,0.099707,2.403677,0.426578
2024-10-22,118.269997,0.723896,1,0.723896,0.099707,2.403677,0.295561
2024-10-23,118.849998,0.490405,1,1.217851,0.099707,2.403677,0.546363
2024-10-24,118.779999,-0.058898,1,1.158236,0.099707,2.403677,0.334875
2024-10-25,119.320000,0.454623,1,1.618124,0.099707,2.403677,0.422098
2024-10-28,120.180000,0.720751,1,2.350538,0.099707,2.403677,0.639368
2024-10-29,119.690002,-0.407720,1,1.933235,0.099707,2.403677,0.813404
2024-10-30,118.839996,-0.710173,1,1.209332,0.099707,2.403677,0.303275
2024-10-31,128.429993,8.069671,1,9.376592,0.099707,2.403677,0.399102


In [46]:
import yfinance as yf
import pandas as pd

# Define tickers and results dictionary
tickers = ['^VIX', '^TNX', '^IRX', 'VXX', 'UVXY', 'MTUM', 
           'SPY', 'QQQ', 'QQQM', 'VOOG', 'VGT', 'XLU',
           'GLD', 'IAU', 'PHYS', 'TLT', 'TMF', 'SHY', 
           'EDV', 'ZROZ', 'GOVT', 'AAPL', 'SFM']
results = {}

# Target return, e.g., risk-free rate
target_return = 0.0

# Loop through each ticker
for ticker in tickers:
    data = yf.download(ticker, start="2022-01-01", end="2024-12-31", interval="1d")

    # Calculate moving averages
    data['3_SMA'] = data['Close'].rolling(window=3).mean()
    data['5_SMA'] = data['Close'].rolling(window=5).mean()

    # Market Trend and price change calculations
    data['Market_Trend'] = 0
    data.loc[data['3_SMA'] > data['5_SMA'], 'Market_Trend'] = 1
    data.loc[data['3_SMA'] < data['5_SMA'], 'Market_Trend'] = -1
    data['Price_Change%'] = data['Close'].pct_change() * 100 
    data['Cumulative_Returns'] = (1 + data['Price_Change%'] / 100).cumprod()
    data['Max_Drawdown'] = (data['Cumulative_Returns'] / data['Cumulative_Returns'].cummax() - 1)

    # Calculate Sortino Ratio (rolling)
    rolling_window = 14  # Define the rolling window size (days)
    average_return = data['Price_Change%'].rolling(window=rolling_window).mean()
    downside_returns = data['Price_Change%'][data['Price_Change%'] < target_return].rolling(window=rolling_window)
    downside_deviation = (downside_returns.apply(lambda x: (x ** 2).mean() ** 0.5))

    # Calculate Sortino Ratio for each window
    data['Sortino Ratio'] = (average_return - target_return) / downside_deviation
    data['Sortino Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities

    # Maximum drawdown calculations
    max_drawdown_value = data['Max_Drawdown'].min()

    # Calculate Calmar Ratio using rolling average return
    annualized_return = ((1 + average_return / 100) ** 252) - 1
    data['Calmar Ratio'] = annualized_return / abs(max_drawdown_value)  # This is typically calculated annually

    # Store results
    trend_indicator = data[['Close', 'Price_Change%', 'Market_Trend', 'Sortino Ratio', 'Calmar Ratio']].dropna()
    trend_indicator.columns = [f'{ticker} Close',
                               f'{ticker} daily move%',
                               f'{ticker} trend signal',
                               f'{ticker} Sortino Ratio',
                               f'{ticker} Calmar Ratio']
    
    results[ticker] = trend_indicator

# Display the last few entries for each ticker
for ticker in tickers:
    display(results[ticker].tail(10))


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,^VIX Close,^VIX daily move%,^VIX trend signal,^VIX Sortino Ratio,^VIX Calmar Ratio
Date,,,,,
2024-10-11,20.459999,-2.245586,-1,0.462749,261.814111
2024-10-14,19.700001,-3.714557,-1,0.489047,236.718487
2024-10-16,19.580000,-5.135656,-1,0.471528,227.066141
2024-10-17,19.110001,-2.400405,-1,0.260566,22.394898
2024-10-18,18.030001,-5.651491,-1,0.225660,9.619156
2024-10-22,18.200001,-0.925422,-1,-0.021366,-0.258070
2024-10-24,19.080000,-0.831600,1,0.033461,0.513930
2024-10-28,19.799999,-2.606988,1,-0.140130,-1.053010
2024-10-29,19.340000,-2.323228,1,-0.132152,-1.029798


,^TNX Close,^TNX daily move%,^TNX trend signal,^TNX Sortino Ratio,^TNX Calmar Ratio
Date,,,,,
2024-09-24,3.736,-0.080236,1,-0.052726,-0.493013
2024-09-27,3.749,-1.107882,1,0.092605,1.087636
2024-10-01,3.743,-1.551815,-1,0.156085,2.070880
2024-10-11,4.073,-0.561529,1,0.554221,13.684442
2024-10-15,4.038,-1.464127,-1,0.426792,8.523254
2024-10-16,4.016,-0.544832,-1,0.406947,6.880046
2024-10-18,4.073,-0.561529,-1,0.555563,9.230652
2024-10-24,4.200,-0.990106,1,0.430758,6.064927
2024-10-29,4.274,-0.093495,1,0.405279,5.405670


,^IRX Close,^IRX daily move%,^IRX trend signal,^IRX Sortino Ratio,^IRX Calmar Ratio
Date,,,,,
2024-10-15,4.515,-0.066407,-1,0.055564,0.411032
2024-10-16,4.503,-0.265783,-1,0.069351,0.521221
2024-10-18,4.513,-0.044294,-1,0.054238,0.216004
2024-10-22,4.500,-0.398409,1,0.098287,0.396203
2024-10-24,4.487,-0.399559,-1,-0.078640,-0.235324
2024-10-28,4.482,-0.620850,-1,-0.233907,-0.570883
2024-10-29,4.480,-0.044620,-1,-0.282962,-0.671274
2024-10-30,4.463,-0.379469,-1,-0.331833,-0.801367
2024-10-31,4.432,-0.694592,-1,-0.375214,-0.967316


,VXX Close,VXX daily move%,VXX trend signal,VXX Sortino Ratio,VXX Calmar Ratio
Date,,,,,
2024-10-11,54.070000,-1.169803,-1,0.319354,11.393381
2024-10-14,52.150002,-3.550949,-1,0.267402,5.455992
2024-10-16,53.330002,-0.965639,-1,0.291831,6.447527
2024-10-17,52.450001,-1.650105,1,0.132012,1.349390
2024-10-18,51.209999,-2.364160,-1,0.124249,1.290129
2024-10-22,51.150002,-0.389478,-1,-0.062097,-0.353938
2024-10-24,52.810001,-1.693968,1,0.012637,0.091596
2024-10-28,52.709999,-4.666308,1,-0.122498,-0.599988
2024-10-29,52.480000,-0.436349,1,-0.066269,-0.381689


...

,EDV Close,EDV daily move%,EDV trend signal,EDV Sortino Ratio,EDV Calmar Ratio
Date,,,,,
2024-10-09,75.300003,-0.698930,-1,-0.541085,-1.364674
2024-10-10,74.720001,-0.770255,-1,-0.555208,-1.384750
2024-10-11,74.309998,-0.548720,-1,-0.641814,-1.403920
2024-10-14,74.180000,-0.174939,-1,-0.656118,-1.409116
2024-10-17,74.480003,-2.257210,1,-0.492822,-1.348491
2024-10-21,72.680000,-2.456049,-1,-0.529848,-1.500038
2024-10-23,72.580002,-0.288500,-1,-0.395422,-1.314001
2024-10-25,72.650002,-0.792028,1,-0.242003,-0.966107
2024-10-28,72.410004,-0.330348,1,-0.270374,-1.037920


,ZROZ Close,ZROZ daily move%,ZROZ trend signal,ZROZ Sortino Ratio,ZROZ Calmar Ratio
Date,,,,,
2024-10-09,77.760002,-0.588084,-1,-0.500576,-1.333912
2024-10-10,76.989998,-0.990232,-1,-0.500328,-1.351170
2024-10-11,76.379997,-0.792312,-1,-0.525825,-1.381103
2024-10-14,76.360001,-0.026180,-1,-0.543421,-1.390794
2024-10-17,76.680000,-2.541942,1,-0.442816,-1.313892
2024-10-21,74.589996,-2.953427,-1,-0.503938,-1.492113
2024-10-23,74.650002,-0.440120,-1,-0.364563,-1.297369
2024-10-25,74.790001,-0.716847,1,-0.209433,-0.941604
2024-10-28,74.580002,-0.280785,1,-0.242465,-1.010516


,GOVT Close,GOVT daily move%,GOVT trend signal,GOVT Sortino Ratio,GOVT Calmar Ratio
Date,,,,,
2024-10-10,23.010000,-0.043441,-1,-0.509194,-1.821257
2024-10-14,23.000000,-0.086883,-1,-0.534076,-1.880703
2024-10-17,23.020000,-0.432528,1,-0.494572,-1.790152
2024-10-21,22.910000,-0.564241,-1,-0.489956,-1.943077
2024-10-22,22.900000,-0.043650,-1,-0.445150,-1.797119
2024-10-23,22.860001,-0.174668,-1,-0.397265,-1.647026
2024-10-25,22.860001,-0.174668,-1,-0.184572,-0.821265
2024-10-28,22.830000,-0.131237,-1,-0.239074,-1.044936
2024-10-30,22.820000,-0.087568,-1,-0.178127,-0.784770


,AAPL Close,AAPL daily move%,AAPL trend signal,AAPL Sortino Ratio,AAPL Calmar Ratio
Date,,,,,
2024-10-07,221.690002,-2.253087,-1,0.114917,1.794073
2024-10-10,229.039993,-0.217827,1,0.023961,0.310592
2024-10-11,227.550003,-0.650537,1,0.028957,0.380888
2024-10-16,231.779999,-0.885186,1,0.107361,1.430516
2024-10-22,235.860001,-0.262177,1,0.217795,3.482893
2024-10-23,230.759995,-2.162302,1,0.117473,1.726271
2024-10-24,230.570007,-0.082331,-1,0.088487,1.227544
2024-10-30,230.100006,-1.527792,1,0.025902,0.333548
2024-10-31,225.910004,-1.820948,-1,-0.032992,-0.354125


,SFM Close,SFM daily move%,SFM trend signal,SFM Sortino Ratio,SFM Calmar Ratio
Date,,,,,
2024-09-27,107.889999,-0.534712,1,0.351156,8.798256
2024-10-02,110.690002,-0.252319,1,0.396387,10.991342
2024-10-09,113.800003,-1.138040,1,0.349109,8.941849
2024-10-10,112.930000,-0.764502,1,0.303663,6.972020
2024-10-14,115.320000,-0.688945,1,0.271092,5.818926
2024-10-17,116.650002,-0.222395,1,0.374435,9.294214
2024-10-18,115.800003,-0.728674,1,0.229337,4.119756
2024-10-24,118.779999,-0.058898,1,0.362324,4.521717
2024-10-29,119.690002,-0.407720,1,0.420310,4.456227


In [48]:
import yfinance as yf
import pandas as pd

# List of tickers to analyze
tickers = ['^VIX', '^TNX', '^IRX',
           'VXX', 'UVXY',
           'MTUM', 
           'SPY', 'QQQ', 'QQQM', 'VOOG', 'VGT', 'XLU',
           'GLD', 'IAU', 'PHYS',
           'TLT', 'TMF', 'SHY', 'EDV', 'ZROZ', 'GOVT',
           'AAPL', 'SFM']

# Initialize a dictionary to store results for each ticker
results = {}

# Define a target return, for instance, the risk-free rate
target_return = 0.0  # You can set this to your desired target return like the risk-free rate
rolling_window = 14  # Define the window size for rolling calculations (14 days)

# Loop through each ticker
for ticker in tickers:
    # Download data for each ticker
    data = yf.download(ticker, start="2022-01-01", end="2024-12-31", interval="1d")

    # Calculate moving averages
    data['3_SMA'] = data['Close'].rolling(window=3).mean()
    data['5_SMA'] = data['Close'].rolling(window=5).mean()
    data['20_SMA'] = data['Close'].rolling(window=20).mean()
    data['50_SMA'] = data['Close'].rolling(window=50).mean()

    # Determine Market Trend
    data['Market_Trend'] = 0
    data.loc[data['3_SMA'] > data['5_SMA'], 'Market_Trend'] = 1
    data.loc[data['3_SMA'] < data['5_SMA'], 'Market_Trend'] = -1

    # Calculate the Price Change %
    data['Price_Change%'] = data['Close'].pct_change() * 100 

    # Cumulative returns for Maximum Drawdown Calculation
    data['Cumulative_Returns'] = (1 + data['Price_Change%'] / 100).cumprod()

    # Rolling Maximum Drawdown Calculation
    data['Rolling_Peak'] = data['Cumulative_Returns'].cummax()
    data['Rolling_Drawdown'] = (data['Cumulative_Returns'] - data['Rolling_Peak']) / data['Rolling_Peak']
    data['Max_Drawdown'] = data['Rolling_Drawdown'].min()

    # Calculate Sortino Ratio (rolling)
    average_return = data['Price_Change%'].rolling(window=rolling_window).mean()
    downside_returns = data['Price_Change%'][data['Price_Change%'] < target_return].rolling(window=rolling_window)
    downside_deviation = downside_returns.apply(lambda x: (x ** 2).mean() ** 0.5)

    # Calculate Sortino Ratio for each window
    data['Sortino Ratio'] = (average_return - target_return) / downside_deviation
    data['Sortino Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities

    # Calculate Calmar Ratio (rolling)
    annualized_return = ((1 + average_return / 100) ** 252) - 1
    data['Calmar Ratio'] = annualized_return / abs(data['Max_Drawdown'])  # Calculate Calmar part using overall max drawdown

    # Calculate Sharpe Ratio using rolling returns
    risk_free_rate = 0.0  # Set your risk-free rate appropriately
    rolling_average_return = data['Price_Change%'].rolling(window=rolling_window).mean()
    rolling_std_dev_returns = data['Price_Change%'].rolling(window=rolling_window).std()

    data['Sharpe Ratio'] = (rolling_average_return - risk_free_rate) / rolling_std_dev_returns
    data['Sharpe Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities

    # Prepare the trend indicator DataFrame
    trend_indicator = data[['Close', 'Price_Change%', 'Market_Trend', 'Sortino Ratio', 'Calmar Ratio', 'Sharpe Ratio']].dropna()
    trend_indicator.columns = [f'{ticker} Close',
                               f'{ticker} daily move%',
                               f'{ticker} trend signal',
                               f'{ticker} Sortino Ratio',
                               f'{ticker} Calmar Ratio',
                               f'{ticker} Sharpe Ratio']
    
    # Store results
    results[ticker] = trend_indicator

# Display the results for each ticker
for ticker in tickers:
    print(f"Results for {ticker}:")
    display(results[ticker].tail(10))  # Display the last 10 entries


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Results for ^VIX:


,^VIX Close,^VIX daily move%,^VIX trend signal,^VIX Sortino Ratio,^VIX Calmar Ratio,^VIX Sharpe Ratio
Date,,,,,,
2024-10-11,20.459999,-2.245586,-1,0.462749,261.814111,0.271554
2024-10-14,19.700001,-3.714557,-1,0.489047,236.718487,0.265165
2024-10-16,19.580000,-5.135656,-1,0.471528,227.066141,0.254803
2024-10-17,19.110001,-2.400405,-1,0.260566,22.394898,0.145510
2024-10-18,18.030001,-5.651491,-1,0.225660,9.619156,0.102682
2024-10-22,18.200001,-0.925422,-1,-0.021366,-0.258070,-0.011413
2024-10-24,19.080000,-0.831600,1,0.033461,0.513930,0.019095
2024-10-28,19.799999,-2.606988,1,-0.140130,-1.053010,-0.126096
2024-10-29,19.340000,-2.323228,1,-0.132152,-1.029798,-0.121108


Results for ^TNX:


,^TNX Close,^TNX daily move%,^TNX trend signal,^TNX Sortino Ratio,^TNX Calmar Ratio,^TNX Sharpe Ratio
Date,,,,,,
2024-09-24,3.736,-0.080236,1,-0.052726,-0.493013,-0.068106
2024-09-27,3.749,-1.107882,1,0.092605,1.087636,0.115063
2024-10-01,3.743,-1.551815,-1,0.156085,2.070880,0.180595
2024-10-11,4.073,-0.561529,1,0.554221,13.684442,0.495094
2024-10-15,4.038,-1.464127,-1,0.426792,8.523254,0.355383
2024-10-16,4.016,-0.544832,-1,0.406947,6.880046,0.306380
2024-10-18,4.073,-0.561529,-1,0.555563,9.230652,0.364643
2024-10-24,4.200,-0.990106,1,0.430758,6.064927,0.339859
2024-10-29,4.274,-0.093495,1,0.405279,5.405670,0.314994


...

,GOVT Close,GOVT daily move%,GOVT trend signal,GOVT Sortino Ratio,GOVT Calmar Ratio,GOVT Sharpe Ratio
Date,,,,,,
2024-10-10,23.010000,-0.043441,-1,-0.509194,-1.821257,-0.590638
2024-10-14,23.000000,-0.086883,-1,-0.534076,-1.880703,-0.623859
2024-10-17,23.020000,-0.432528,1,-0.494572,-1.790152,-0.517447
2024-10-21,22.910000,-0.564241,-1,-0.489956,-1.943077,-0.527958
2024-10-22,22.900000,-0.043650,-1,-0.445150,-1.797119,-0.479021
2024-10-23,22.860001,-0.174668,-1,-0.397265,-1.647026,-0.441261
2024-10-25,22.860001,-0.174668,-1,-0.184572,-0.821265,-0.239294
2024-10-28,22.830000,-0.131237,-1,-0.239074,-1.044936,-0.318299
2024-10-30,22.820000,-0.087568,-1,-0.178127,-0.784770,-0.238928


Results for AAPL:


,AAPL Close,AAPL daily move%,AAPL trend signal,AAPL Sortino Ratio,AAPL Calmar Ratio,AAPL Sharpe Ratio
Date,,,,,,
2024-10-07,221.690002,-2.253087,-1,0.114917,1.794073,0.102285
2024-10-10,229.039993,-0.217827,1,0.023961,0.310592,0.024983
2024-10-11,227.550003,-0.650537,1,0.028957,0.380888,0.030430
2024-10-16,231.779999,-0.885186,1,0.107361,1.430516,0.093778
2024-10-22,235.860001,-0.262177,1,0.217795,3.482893,0.245605
2024-10-23,230.759995,-2.162302,1,0.117473,1.726271,0.126144
2024-10-24,230.570007,-0.082331,-1,0.088487,1.227544,0.094962
2024-10-30,230.100006,-1.527792,1,0.025902,0.333548,0.035697
2024-10-31,225.910004,-1.820948,-1,-0.032992,-0.354125,-0.038546


Results for SFM:


,SFM Close,SFM daily move%,SFM trend signal,SFM Sortino Ratio,SFM Calmar Ratio,SFM Sharpe Ratio
Date,,,,,,
2024-09-27,107.889999,-0.534712,1,0.351156,8.798256,0.327548
2024-10-02,110.690002,-0.252319,1,0.396387,10.991342,0.364220
2024-10-09,113.800003,-1.138040,1,0.349109,8.941849,0.421032
2024-10-10,112.930000,-0.764502,1,0.303663,6.972020,0.354060
2024-10-14,115.320000,-0.688945,1,0.271092,5.818926,0.318312
2024-10-17,116.650002,-0.222395,1,0.374435,9.294214,0.460314
2024-10-18,115.800003,-0.728674,1,0.229337,4.119756,0.298962
2024-10-24,118.779999,-0.058898,1,0.362324,4.521717,0.324675
2024-10-29,119.690002,-0.407720,1,0.420310,4.456227,0.363371


In [50]:
import yfinance as yf
import pandas as pd

# List of tickers to analyze
tickers = ['^VIX', '^TNX', '^IRX',
           'VXX', 'UVXY',
           'MTUM', 
           'SPY', 'QQQ', 'QQQM', 'VOOG', 'VGT', 'XLU',
           'GLD', 'IAU', 'PHYS',
           'TLT', 'TMF', 'SHY', 'EDV', 'ZROZ', 'GOVT',
           'AAPL', 'SFM']

# Initialize a dictionary to store results for each ticker
results = {}

# Define a target return (for instance, the risk-free rate)
target_return = 0.0  # You can adjust this as needed
rolling_window = 14  # Define the window size for rolling calculations

# Loop through each ticker
for ticker in tickers:
    # Download data for each ticker
    data = yf.download(ticker, start="2022-01-01", end="2024-12-31", interval="1d")

    # Calculate moving averages
    data['3_SMA'] = data['Close'].rolling(window=3).mean()
    data['5_SMA'] = data['Close'].rolling(window=5).mean()
    data['20_SMA'] = data['Close'].rolling(window=20).mean()
    data['50_SMA'] = data['Close'].rolling(window=50).mean()

    # Determine Market Trend
    data['Market_Trend'] = 0
    data.loc[data['3_SMA'] > data['5_SMA'], 'Market_Trend'] = 1
    data.loc[data['3_SMA'] < data['5_SMA'], 'Market_Trend'] = -1

    # Calculate the Price Change %
    data['Price_Change%'] = data['Close'].pct_change() * 100 

    # Cumulative returns for Maximum Drawdown Calculation
    data['Cumulative_Returns'] = (1 + data['Price_Change%'] / 100).cumprod()

    # Rolling Maximum Drawdown Calculation
    data['Rolling_Peak'] = data['Cumulative_Returns'].cummax()
    data['Rolling_Drawdown'] = (data['Cumulative_Returns'] - data['Rolling_Peak']) / data['Rolling_Peak']

    # Initialize columns for cumulative change calculation
    data['Cumulative_Change%'] = 0.0
    current_base_price = None
    current_trend = None
    previous_close = None

    for index in data.index:
        market_trend = data.at[index, 'Market_Trend']
        close_price = data.at[index, 'Close']

        if market_trend != current_trend:
            if previous_close is not None:
                data.at[index, 'Cumulative_Change%'] = ((close_price - previous_close) / previous_close) * 100
            
            current_base_price = previous_close
        else:
            if current_base_price is not None:
                data.at[index, 'Cumulative_Change%'] = ((close_price - current_base_price) / current_base_price) * 100
        
        previous_close = close_price
        current_trend = market_trend

    # Prepare DataFrame with key indicators
    trend_indicator = data[['Close', 'Price_Change%', 'Market_Trend', 'Cumulative_Change%', 'Rolling_Drawdown']].dropna()
    trend_indicator.columns = [f'{ticker} Close',
                               f'{ticker} daily move%',
                               f'{ticker} trend signal',
                               f'{ticker} cumulative change since signal',
                               f'{ticker} Rolling Max Drawdown']
    
    # Calculate Sortino Ratio (rolling)
    average_return = data['Price_Change%'].rolling(window=rolling_window).mean()
    downside_returns = data['Price_Change%'][data['Price_Change%'] < target_return].rolling(window=rolling_window)
    downside_deviation = downside_returns.apply(lambda x: (x ** 2).mean() ** 0.5)

    # Calculate Sortino Ratio for each window
    data['Sortino Ratio'] = (average_return - target_return) / downside_deviation
    data['Sortino Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities

    # Calculate Calmar Ratio using rolling average return
    annualized_return = ((1 + average_return / 100) ** 252) - 1
    data['Calmar Ratio'] = annualized_return / abs(data['Rolling_Drawdown'].min())  # Use current rolling drawdown for calculation

    # Calculate Sharpe Ratio using rolling returns
    risk_free_rate = 0.0  # Set your risk-free rate appropriately
    rolling_average_return = data['Price_Change%'].rolling(window=rolling_window).mean()
    rolling_std_dev_returns = data['Price_Change%'].rolling(window=rolling_window).std()

    data['Sharpe Ratio'] = (rolling_average_return - risk_free_rate) / rolling_std_dev_returns
    data['Sharpe Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities

    # Add new columns to trend indicator DataFrame
    trend_indicator[f'{ticker} Sortino Ratio'] = data['Sortino Ratio']
    trend_indicator[f'{ticker} Calmar Ratio'] = data['Calmar Ratio']
    trend_indicator[f'{ticker} Sharpe Ratio'] = data['Sharpe Ratio']

    results[ticker] = trend_indicator

# Function to get the current risk-free rate from the results dictionary
def get_risk_free_rate():
    # This implementation assumes ^TNX ticker is available in results
    treasury_yield = results["^TNX"].iloc[-1]['^TNX Close']
    return treasury_yield / 100

# Get the risk-free rate and convert from annual to daily
risk_free_rate = get_risk_free_rate() / 252 

# Calculate and append Sharpe Ratio to each trend_indicator DataFrame
for ticker in tickers:
    if ticker in results:
        average_return = results[ticker][f'{ticker} daily move%'].rolling(window=7).mean()
        rolling_std_dev_returns = results[ticker][f'{ticker} daily move%'].rolling(window=7).std()

        sharpe_ratio = (average_return - risk_free_rate) / rolling_std_dev_returns
        results[ticker][f'{ticker} Sharpe Ratio'] = sharpe_ratio

# Display the results with Sharpe Ratio, Calmar Ratio and Rolling Max Drawdown for each ticker
for ticker in tickers:
    print(f"Results for {ticker}:")
    display(results[ticker].tail(10))  # Display the last 10 entries


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Results for ^VIX:


,^VIX Close,^VIX daily move%,^VIX trend signal,^VIX cumulative change since signal,^VIX Rolling Max Drawdown,^VIX Sortino Ratio,^VIX Calmar Ratio,^VIX Sharpe Ratio
Date,,,,,,,,
2024-10-21,18.370001,1.885747,-1,-12.231244,-0.523723,NaN,-0.449523,-0.468761
2024-10-22,18.200001,-0.925422,-1,-13.043476,-0.528131,-0.021366,-0.258070,-0.418557
2024-10-23,19.240000,5.714280,1,5.714280,-0.501167,NaN,-0.729719,-0.054876
2024-10-24,19.080000,-0.831600,1,4.835160,-0.505315,0.033461,0.513930,-0.264946
2024-10-25,20.330000,6.551363,1,11.703292,-0.472906,NaN,-1.222882,0.141239
2024-10-28,19.799999,-2.606988,1,8.791200,-0.486648,-0.140130,-1.053010,0.133777
2024-10-29,19.340000,-2.323228,1,6.263733,-0.498574,-0.132152,-1.029798,0.283402
2024-10-30,20.350000,5.222338,1,11.813184,-0.472388,NaN,-0.388920,0.378051
2024-10-31,23.160000,13.808351,1,27.252741,-0.399533,NaN,17.931816,0.611832


Results for ^TNX:


,^TNX Close,^TNX daily move%,^TNX trend signal,^TNX cumulative change since signal,^TNX Rolling Max Drawdown,^TNX Sortino Ratio,^TNX Calmar Ratio,^TNX Sharpe Ratio
Date,,,,,,,,
2024-10-21,4.182,2.676165,1,2.676165,-0.161588,NaN,23.783950,0.201552
2024-10-22,4.204,0.526060,1,3.216304,-0.157177,NaN,21.013697,0.313591
2024-10-23,4.242,0.903904,1,4.149279,-0.149559,NaN,17.677393,0.339625
2024-10-24,4.200,-0.990106,1,3.118091,-0.157979,0.430758,6.064927,0.412289
2024-10-25,4.232,0.761906,1,3.903755,-0.151564,NaN,5.443832,0.585109
2024-10-28,4.278,1.086957,1,5.033143,-0.142342,NaN,7.061155,0.527022
2024-10-29,4.274,-0.093495,1,4.934943,-0.143143,0.405279,5.405670,0.617507
2024-10-30,4.266,-0.187187,1,4.738518,-0.144747,0.334471,4.055232,0.386349
2024-10-31,4.284,0.421944,1,5.180456,-0.141139,NaN,5.541829,0.368572


...

,GOVT Close,GOVT daily move%,GOVT trend signal,GOVT cumulative change since signal,GOVT Rolling Max Drawdown,GOVT Sortino Ratio,GOVT Calmar Ratio,GOVT Sharpe Ratio
Date,,,,,,,,
2024-10-21,22.910000,-0.564241,-1,-0.564241,-0.125906,-0.489956,-1.943077,-0.182166
2024-10-22,22.900000,-0.043650,-1,-0.607644,-0.126288,-0.445150,-1.797119,-0.220693
2024-10-23,22.860001,-0.174668,-1,-0.781251,-0.127814,-0.397265,-1.647026,-0.256271
2024-10-24,22.900000,0.174974,-1,-0.607644,-0.126288,NaN,-0.932101,-0.440068
2024-10-25,22.860001,-0.174668,-1,-0.781251,-0.127814,-0.184572,-0.821265,-0.606208
2024-10-28,22.830000,-0.131237,-1,-0.911463,-0.128958,-0.239074,-1.044936,-0.497473
2024-10-29,22.840000,0.043803,-1,-0.868059,-0.128577,NaN,-0.745667,-0.536223
2024-10-30,22.820000,-0.087568,-1,-0.954866,-0.129340,-0.178127,-0.784770,-0.440030
2024-10-31,22.820000,0.000000,-1,-0.954866,-0.129340,NaN,-0.822981,-0.385996


Results for AAPL:


,AAPL Close,AAPL daily move%,AAPL trend signal,AAPL cumulative change since signal,AAPL Rolling Max Drawdown,AAPL Sortino Ratio,AAPL Calmar Ratio,AAPL Sharpe Ratio
Date,,,,,,,,
2024-10-21,236.479996,0.629785,1,3.023439,0.000000,NaN,4.137184,0.478772
2024-10-22,235.860001,-0.262177,1,2.753336,-0.002622,0.217795,3.482893,0.575587
2024-10-23,230.759995,-2.162302,1,0.531498,-0.024188,0.117473,1.726271,-0.022742
2024-10-24,230.570007,-0.082331,-1,-0.082331,-0.024991,0.088487,1.227544,-0.179539
2024-10-25,231.410004,0.364313,-1,0.281682,-0.021439,NaN,3.936604,-0.016980
2024-10-28,233.399994,0.859941,-1,1.144046,-0.013024,NaN,2.770973,0.073400
2024-10-29,233.669998,0.115683,1,0.115683,-0.011883,NaN,1.295219,-0.076951
2024-10-30,230.100006,-1.527792,1,-1.413877,-0.026979,0.025902,0.333548,-0.358262
2024-10-31,225.910004,-1.820948,-1,-1.820948,-0.044697,-0.032992,-0.354125,-0.506699


Results for SFM:


,SFM Close,SFM daily move%,SFM trend signal,SFM cumulative change since signal,SFM Rolling Max Drawdown,SFM Sortino Ratio,SFM Calmar Ratio,SFM Sharpe Ratio
Date,,,,,,,,
2024-10-21,117.419998,1.398959,-1,1.398959,0.000000,NaN,5.346115,0.426578
2024-10-22,118.269997,0.723896,1,0.723896,0.000000,NaN,6.932574,0.295561
2024-10-23,118.849998,0.490405,1,1.217851,0.000000,NaN,7.245904,0.546363
2024-10-24,118.779999,-0.058898,1,1.158236,-0.000589,0.362324,4.521717,0.334875
2024-10-25,119.320000,0.454623,1,1.618124,0.000000,NaN,4.940532,0.422098
2024-10-28,120.180000,0.720751,1,2.350538,0.000000,NaN,3.546495,0.639368
2024-10-29,119.690002,-0.407720,1,1.933235,-0.004077,0.420310,4.456227,0.813404
2024-10-30,118.839996,-0.710173,1,1.209332,-0.011150,0.428956,4.528765,0.303275
2024-10-31,128.429993,8.069671,1,9.376592,0.000000,NaN,16.177376,0.399102


In [ ]:
import yfinance as yf
import pandas as pd

# List of tickers to analyze
tickers = ['^VIX', '^TNX', '^IRX',
           'VXX', 'UVXY',
           'MTUM', 
           'SPY', 'QQQ', 'QQQM', 'VOOG', 'VGT', 'XLU',
           'GLD', 'IAU', 'PHYS',
           'TLT', 'TMF', 'SHY', 'EDV', 'ZROZ', 'GOVT',
           'AAPL', 'SFM']

# Initialize a dictionary to store results for each ticker
results = {}

# Define a target return (for instance, the risk-free rate)
target_return = 0.0  # Adjust this as needed
rolling_window = 14  # Define the window size for rolling calculations

# Loop through each ticker
for ticker in tickers:
    # ** Data Downloading Section **
    # Download data for each ticker
    data = yf.download(ticker, start="2022-01-01", end="2024-12-31", interval="1d")

    # ** Moving Averages Section **
    # Calculate moving averages
    data['3_SMA'] = data['Close'].rolling(window=3).mean()
    data['5_SMA'] = data['Close'].rolling(window=5).mean()
    data['20_SMA'] = data['Close'].rolling(window=20).mean()
    data['50_SMA'] = data['Close'].rolling(window=50).mean()

    # ** Market Trend Section **
    # Determine Market Trend
    data['Market_Trend'] = 0
    data.loc[data['3_SMA'] > data['5_SMA'], 'Market_Trend'] = 1
    data.loc[data['3_SMA'] < data['5_SMA'], 'Market_Trend'] = -1

    # ** Price Change Calculations Section **
    # Calculate the Price Change %
    data['Price_Change%'] = data['Close'].pct_change() * 100 
    data['Cumulative_Returns'] = (1 + data['Price_Change%'] / 100).cumprod()

    # ** Rolling Maximum Drawdown Calculation Section **
    data['Rolling_Peak'] = data['Cumulative_Returns'].cummax()
    data['Rolling_Drawdown'] = (data['Cumulative_Returns'] - data['Rolling_Peak']) / data['Rolling_Peak']

    # ** Cumulative Change Calculation Section **
    data['Cumulative_Change%'] = 0.0
    current_base_price = None
    current_trend = None
    previous_close = None

    for index in data.index:
        market_trend = data.at[index, 'Market_Trend']
        close_price = data.at[index, 'Close']

        if market_trend != current_trend:
            if previous_close is not None:
                data.at[index, 'Cumulative_Change%'] = ((close_price - previous_close) / previous_close) * 100
            
            current_base_price = previous_close
        else:
            if current_base_price is not None:
                data.at[index, 'Cumulative_Change%'] = ((close_price - current_base_price) / current_base_price) * 100
        
        previous_close = close_price
        current_trend = market_trend

    # ** Preparing the Trend Indicator DataFrame **
    trend_indicator = data[['Close', 'Price_Change%', 'Market_Trend', 'Cumulative_Change%', 'Rolling_Drawdown']].dropna()
    trend_indicator.columns = [f'{ticker} Close',
                               f'{ticker} daily move%',
                               f'{ticker} trend signal',
                               f'{ticker} cumulative change since signal',
                               f'{ticker} Rolling Max Drawdown']
    
    # ** Sortino Ratio Calculation Section **
    average_return = data['Price_Change%'].rolling(window=rolling_window).mean()
    downside_returns = data['Price_Change%'][data['Price_Change%'] < target_return].rolling(window=rolling_window)
    downside_deviation = downside_returns.apply(lambda x: (x ** 2).mean() ** 0.5)

    # Calculate Sortino Ratio
    data['Sortino Ratio'] = (average_return - target_return) / downside_deviation
    data['Sortino Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities

    # ** Calmar Ratio Calculation Section **
    annualized_return = ((1 + average_return / 100) ** 252) - 1
    data['Calmar Ratio'] = annualized_return / abs(data['Rolling_Drawdown'].min())  # Use current rolling drawdown

    # ** Sharpe Ratio Calculation Section **
    # Function to get the current risk-free rate from the results dictionary
    def get_risk_free_rate():
        treasury_yield = results["^TNX"].iloc[-1]['^TNX Close']
        return treasury_yield / 100

    # Get the risk-free rate converted from annual to daily
    risk_free_rate = get_risk_free_rate() / 252 
    
    #risk_free_rate = 0.0  # Set your risk-free rate appropriately
    rolling_average_return = data['Price_Change%'].rolling(window=rolling_window).mean()
    rolling_std_dev_returns = data['Price_Change%'].rolling(window=rolling_window).std()

    data['Sharpe Ratio'] = (rolling_average_return - risk_free_rate) / rolling_std_dev_returns
    data['Sharpe Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities

    # ** Add New Columns to Trend Indicator DataFrame **
    trend_indicator[f'{ticker} Sortino Ratio'] = data['Sortino Ratio']
    trend_indicator[f'{ticker} Calmar Ratio'] = data['Calmar Ratio']
    trend_indicator[f'{ticker} Sharpe Ratio'] = data['Sharpe Ratio']

    # Store the results for this ticker
    results[ticker] = trend_indicator

# ** Display Results Section **
for ticker in tickers:
    print(f"Results for {ticker}:")
    display(results[ticker].tail(10))  # Display the last 10 entries


[*********************100%%**********************]  1 of 1 completed


KeyError: '^TNX'

In [56]:
import yfinance as yf
import pandas as pd

# List of tickers to analyze
tickers = ['^VIX', '^TNX', '^IRX',
           'VXX', 'UVXY',
           'MTUM', 
           'SPY', 'QQQ', 'QQQM', 'VOOG', 'VGT', 'XLU',
           'GLD', 'IAU', 'PHYS',
           'TLT', 'TMF', 'SHY', 'EDV', 'ZROZ', 'GOVT',
           'AAPL', 'SFM']

# Initialize a dictionary to store results for each ticker
results = {}

# Define a target return (for instance, the risk-free rate)
target_return = 0.0  # Adjust this as needed
rolling_window = 14  # Define the window size for rolling calculations

# ** Data Processing Loop for Each Ticker **
for ticker in tickers:
    # ** 1. Data Downloading Section **
    try:
        data = yf.download(ticker, start="2022-01-01", end="2024-12-31", interval="1d")

        # Check if the data is empty
        if data.empty:
            print(f"No data found for {ticker}. Skipping.")
            continue

        # ** 2. Moving Averages Section **
        data['3_SMA'] = data['Close'].rolling(window=3).mean()
        data['5_SMA'] = data['Close'].rolling(window=5).mean()
        data['20_SMA'] = data['Close'].rolling(window=20).mean()
        data['50_SMA'] = data['Close'].rolling(window=50).mean()

        # ** 3. Market Trend Section **
        data['Market_Trend'] = 0
        data.loc[data['3_SMA'] > data['5_SMA'], 'Market_Trend'] = 1
        data.loc[data['3_SMA'] < data['5_SMA'], 'Market_Trend'] = -1

        # ** 4. Price Change Calculations Section **
        data['Price_Change%'] = data['Close'].pct_change() * 100 
        data['Cumulative_Returns'] = (1 + data['Price_Change%'] / 100).cumprod()

        # ** 5. Rolling Maximum Drawdown Calculation Section **
        data['Rolling_Peak'] = data['Cumulative_Returns'].cummax()
        data['Rolling_Drawdown'] = (data['Cumulative_Returns'] - data['Rolling_Peak']) / data['Rolling_Peak']

        # ** 6. Cumulative Change Calculation Section **
        data['Cumulative_Change%'] = 0.0
        current_base_price = None
        current_trend = None
        previous_close = None

        for index in data.index:
            market_trend = data.at[index, 'Market_Trend']
            close_price = data.at[index, 'Close']

            if market_trend != current_trend:
                if previous_close is not None:
                    data.at[index, 'Cumulative_Change%'] = ((close_price - previous_close) / previous_close) * 100
                
                current_base_price = previous_close
            else:
                if current_base_price is not None:
                    data.at[index, 'Cumulative_Change%'] = ((close_price - current_base_price) / current_base_price) * 100
            
            previous_close = close_price
            current_trend = market_trend

        # ** 7. Preparing the Trend Indicator DataFrame **
        trend_indicator = data[['Close', 'Price_Change%', 'Market_Trend', 'Cumulative_Change%', 'Rolling_Drawdown']].dropna()
        trend_indicator.columns = [f'{ticker} Close',
                                   f'{ticker} daily move%',
                                   f'{ticker} trend signal',
                                   f'{ticker} cumulative change since signal',
                                   f'{ticker} Rolling Max Drawdown']

        # ** 8. Sortino Ratio Calculation Section **
        average_return = data['Price_Change%'].rolling(window=rolling_window).mean()
        downside_returns = data['Price_Change%'][data['Price_Change%'] < target_return].rolling(window=rolling_window)
        downside_deviation = downside_returns.apply(lambda x: (x ** 2).mean() ** 0.5)

        # Calculate Sortino Ratio
        data['Sortino Ratio'] = (average_return - target_return) / downside_deviation
        data['Sortino Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities

        # ** 9. Calmar Ratio Calculation Section **
        annualized_return = ((1 + average_return / 100) ** 252) - 1
        data['Calmar Ratio'] = annualized_return / abs(data['Rolling_Drawdown'].min())  # Use current rolling drawdown

        # ** 10. Sharpe Ratio Calculation Section **
        trend_indicator[f'{ticker} Sortino Ratio'] = data['Sortino Ratio']
        trend_indicator[f'{ticker} Calmar Ratio'] = data['Calmar Ratio']

        results[ticker] = trend_indicator  # Store the results for this ticker

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# ** Risk-Free Rate Calculation Section **
if "^TNX" in results:  # Ensure ^TNX has been processed successfully
    treasury_yield = results["^TNX"]['^TNX Close']
    risk_free_rate = treasury_yield / 100 / 252  # Convert from annual to daily
else:
    print("Warning: ^TNX results not found. Defaulting risk-free rate to 0.")
    risk_free_rate = 0.0  # Fallback risk-free rate

# ** Calculate Sharpe Ratio for all tickers using the risk-free rate **
for ticker in tickers:
    if ticker in results:
        data = results[ticker]
        rolling_average_return = data[f'{ticker} daily move%'].rolling(window=rolling_window).mean()
        rolling_std_dev_returns = data[f'{ticker} daily move%'].rolling(window=rolling_window).std()

        data[f'{ticker} Sharpe Ratio'] = (rolling_average_return - risk_free_rate) / rolling_std_dev_returns
        data[f'{ticker} Sharpe Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities
        results[ticker] = data  # Update results with Sharpe Ratio

# ** 12. Display Results Section **
for ticker in tickers:
    if ticker in results:  # Only display results for tickers that were processed successfully
        print(f"Results for {ticker}:")
        display(results[ticker].tail(10))  # Display the last 10 entries


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Results for ^VIX:


,^VIX Close,^VIX daily move%,^VIX trend signal,^VIX cumulative change since signal,^VIX Rolling Max Drawdown,^VIX Sortino Ratio,^VIX Calmar Ratio,^VIX Sharpe Ratio
Date,,,,,,,,
2024-10-21,18.370001,1.885747,-1,-12.231244,-0.523723,NaN,-0.449523,-0.021532
2024-10-22,18.200001,-0.925422,-1,-13.043476,-0.528131,-0.021366,-0.258070,-0.011438
2024-10-23,19.240000,5.714280,1,5.714280,-0.501167,NaN,-0.729719,-0.041877
2024-10-24,19.080000,-0.831600,1,4.835160,-0.505315,0.033461,0.513930,0.019068
2024-10-25,20.330000,6.551363,1,11.703292,-0.472906,NaN,-1.222882,-0.169247
2024-10-28,19.799999,-2.606988,1,8.791200,-0.486648,-0.140130,-1.053010,-0.126140
2024-10-29,19.340000,-2.323228,1,6.263733,-0.498574,-0.132152,-1.029798,-0.121151
2024-10-30,20.350000,5.222338,1,11.813184,-0.472388,NaN,-0.388920,-0.029010
2024-10-31,23.160000,13.808351,1,27.252741,-0.399533,NaN,17.931816,0.185701


Results for ^TNX:


,^TNX Close,^TNX daily move%,^TNX trend signal,^TNX cumulative change since signal,^TNX Rolling Max Drawdown,^TNX Sortino Ratio,^TNX Calmar Ratio,^TNX Sharpe Ratio
Date,,,,,,,,
2024-10-21,4.182,2.676165,1,2.676165,-0.161588,NaN,23.783950,0.593306
2024-10-22,4.204,0.526060,1,3.216304,-0.157177,NaN,21.013697,0.562457
2024-10-23,4.242,0.903904,1,4.149279,-0.149559,NaN,17.677393,0.530101
2024-10-24,4.200,-0.990106,1,3.118091,-0.157979,0.430758,6.064927,0.339713
2024-10-25,4.232,0.761906,1,3.903755,-0.151564,NaN,5.443832,0.320709
2024-10-28,4.278,1.086957,1,5.033143,-0.142342,NaN,7.061155,0.373575
2024-10-29,4.274,-0.093495,1,4.934943,-0.143143,0.405279,5.405670,0.314846
2024-10-30,4.266,-0.187187,1,4.738518,-0.144747,0.334471,4.055232,0.257888
2024-10-31,4.284,0.421944,1,5.180456,-0.141139,NaN,5.541829,0.326500


...

,GOVT Close,GOVT daily move%,GOVT trend signal,GOVT cumulative change since signal,GOVT Rolling Max Drawdown,GOVT Sortino Ratio,GOVT Calmar Ratio,GOVT Sharpe Ratio
Date,,,,,,,,
2024-10-21,22.910000,-0.564241,-1,-0.564241,-0.125906,-0.489956,-1.943077,-0.528487
2024-10-22,22.900000,-0.043650,-1,-0.607644,-0.126288,-0.445150,-1.797119,-0.479551
2024-10-23,22.860001,-0.174668,-1,-0.781251,-0.127814,-0.397265,-1.647026,-0.441808
2024-10-24,22.900000,0.174974,-1,-0.607644,-0.126288,NaN,-0.932101,-0.268558
2024-10-25,22.860001,-0.174668,-1,-0.781251,-0.127814,-0.184572,-0.821265,-0.239943
2024-10-28,22.830000,-0.131237,-1,-0.911463,-0.128958,-0.239074,-1.044936,-0.318970
2024-10-29,22.840000,0.043803,-1,-0.868059,-0.128577,NaN,-0.745667,-0.226939
2024-10-30,22.820000,-0.087568,-1,-0.954866,-0.129340,-0.178127,-0.784770,-0.239614
2024-10-31,22.820000,0.000000,-1,-0.954866,-0.129340,NaN,-0.822981,-0.253352


Results for AAPL:


,AAPL Close,AAPL daily move%,AAPL trend signal,AAPL cumulative change since signal,AAPL Rolling Max Drawdown,AAPL Sortino Ratio,AAPL Calmar Ratio,AAPL Sharpe Ratio
Date,,,,,,,,
2024-10-21,236.479996,0.629785,1,3.023439,0.000000,NaN,4.137184,0.279395
2024-10-22,235.860001,-0.262177,1,2.753336,-0.002622,0.217795,3.482893,0.245462
2024-10-23,230.759995,-2.162302,1,0.531498,-0.024188,0.117473,1.726271,0.126018
2024-10-24,230.570007,-0.082331,-1,-0.082331,-0.024991,0.088487,1.227544,0.094836
2024-10-25,231.410004,0.364313,-1,0.281682,-0.021439,NaN,3.936604,0.275149
2024-10-28,233.399994,0.859941,-1,1.144046,-0.013024,NaN,2.770973,0.228308
2024-10-29,233.669998,0.115683,1,0.115683,-0.011883,NaN,1.295219,0.134335
2024-10-30,230.100006,-1.527792,1,-1.413877,-0.026979,0.025902,0.333548,0.035540
2024-10-31,225.910004,-1.820948,-1,-1.820948,-0.044697,-0.032992,-0.354125,-0.038691


Results for SFM:


,SFM Close,SFM daily move%,SFM trend signal,SFM cumulative change since signal,SFM Rolling Max Drawdown,SFM Sortino Ratio,SFM Calmar Ratio,SFM Sharpe Ratio
Date,,,,,,,,
2024-10-21,117.419998,1.398959,-1,1.398959,0.000000,NaN,5.346115,0.343769
2024-10-22,118.269997,0.723896,1,0.723896,0.000000,NaN,6.932574,0.406640
2024-10-23,118.849998,0.490405,1,1.217851,0.000000,NaN,7.245904,0.417505
2024-10-24,118.779999,-0.058898,1,1.158236,-0.000589,0.362324,4.521717,0.324528
2024-10-25,119.320000,0.454623,1,1.618124,0.000000,NaN,4.940532,0.344105
2024-10-28,120.180000,0.720751,1,2.350538,0.000000,NaN,3.546495,0.293781
2024-10-29,119.690002,-0.407720,1,1.933235,-0.004077,0.420310,4.456227,0.363203
2024-10-30,118.839996,-0.710173,1,1.209332,-0.011150,0.428956,4.528765,0.368743
2024-10-31,128.429993,8.069671,1,9.376592,0.000000,NaN,16.177376,0.334452


In [57]:
import yfinance as yf
import pandas as pd

# List of tickers to analyze
tickers = ['^VIX', '^TNX', '^IRX',
           'VXX', 'UVXY',
           'MTUM', 
           'SPY', 'QQQ', 'QQQM', 'VOOG', 'VGT', 'XLU',
           'GLD', 'IAU', 'PHYS',
           'TLT', 'TMF', 'SHY', 'EDV', 'ZROZ', 'GOVT',
           'AAPL', 'SFM']

# Initialize a dictionary to store results for each ticker
results = {}

# Define a target return (for instance, the risk-free rate)
target_return = 0.0  # Adjust this as needed
rolling_window = 14  # Define the window size for rolling calculations

# ** Data Processing Loop for Each Ticker **
for ticker in tickers:
    # ** 1. Data Downloading Section **
    try:
        data = yf.download(ticker, start="2022-01-01", end="2024-12-31", interval="1d")

        # Check if the data is empty
        if data.empty:
            print(f"No data found for {ticker}. Skipping.")
            continue

        # ** 2. Moving Averages Section **
        data['3_SMA'] = data['Close'].rolling(window=3).mean()
        data['5_SMA'] = data['Close'].rolling(window=5).mean()
        data['20_SMA'] = data['Close'].rolling(window=20).mean()
        data['50_SMA'] = data['Close'].rolling(window=50).mean()

        # ** 3. Market Trend Section **
        data['Market_Trend'] = 0
        data.loc[data['3_SMA'] > data['5_SMA'], 'Market_Trend'] = 1
        data.loc[data['3_SMA'] < data['5_SMA'], 'Market_Trend'] = -1

        # ** 4. Price Change Calculations Section **
        data['Price_Change%'] = data['Close'].pct_change() * 100 
        data['Cumulative_Returns'] = (1 + data['Price_Change%'] / 100).cumprod()

        # ** 5. Rolling Maximum Drawdown Calculation Section **
        data['Rolling_Peak'] = data['Cumulative_Returns'].cummax()
        data['Rolling_Drawdown'] = (data['Cumulative_Returns'] - data['Rolling_Peak']) / data['Rolling_Peak']

        # ** 6. Cumulative Change Calculation Section using vectorized operations **
        current_trend_shifted = data['Market_Trend'].shift(1)
        current_base_price = data['Close'].shift(1).where(current_trend_shifted != data['Market_Trend']).ffill()
        data['Cumulative_Change%'] = ((data['Close'] - current_base_price) / current_base_price) * 100

        # ** 7. Preparing the Trend Indicator DataFrame **
        trend_indicator = data[['Close', 'Price_Change%', 'Market_Trend', 'Cumulative_Change%', 'Rolling_Drawdown']].dropna()
        trend_indicator.columns = [f'{ticker} Close',
                                   f'{ticker} daily move%',
                                   f'{ticker} trend signal',
                                   f'{ticker} cumulative change since signal',
                                   f'{ticker} Rolling Max Drawdown']

        # ** 8. Sortino Ratio Calculation Section **
        average_return = data['Price_Change%'].rolling(window=rolling_window).mean()
        downside_returns = data['Price_Change%'][data['Price_Change%'] < target_return].rolling(window=rolling_window)
        downside_deviation = downside_returns.apply(lambda x: (x ** 2).mean() ** 0.5 if not x.empty else 0)

        # Calculate Sortino Ratio
        data['Sortino Ratio'] = (average_return - target_return) / downside_deviation
        data['Sortino Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities

        # ** 9. Calmar Ratio Calculation Section **
        annualized_return = ((1 + average_return / 100) ** 252) - 1
        data['Calmar Ratio'] = annualized_return / abs(data['Rolling_Drawdown'].min())  # Use current rolling drawdown

        # ** 10. Sharpe Ratio Calculation Section **
        trend_indicator[f'{ticker} Sortino Ratio'] = data['Sortino Ratio']
        trend_indicator[f'{ticker} Calmar Ratio'] = data['Calmar Ratio']

        results[ticker] = trend_indicator  # Store the results for this ticker

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# ** Risk-Free Rate Calculation Section **
if "^TNX" in results:  # Ensure ^TNX has been processed successfully
    treasury_yield = results["^TNX"]['^TNX Close']
    risk_free_rate = treasury_yield / 100 / 252  # Convert from annual to daily
else:
    print("Warning: ^TNX results not found. Defaulting risk-free rate to 0.")
    risk_free_rate = 0.0  # Fallback risk-free rate

# ** Calculate Sharpe Ratio for all tickers using the risk-free rate **
for ticker in tickers:
    if ticker in results:
        data = results[ticker]
        rolling_average_return = data[f'{ticker} daily move%'].rolling(window=rolling_window).mean()
        rolling_std_dev_returns = data[f'{ticker} daily move%'].rolling(window=rolling_window).std()

        # Define Sharpe Ratio; replace infinities for stability
        data[f'{ticker} Sharpe Ratio'] = (rolling_average_return - risk_free_rate) / rolling_std_dev_returns
        data[f'{ticker} Sharpe Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities
        results[ticker] = data  # Update results with Sharpe Ratio

# ** 12. Display Results Section **
for ticker in tickers:
    if ticker in results:  # Only display results for tickers that were processed successfully
        print(f"Results for {ticker}:")
        display(results[ticker].tail(10))  # Display the last 10 entries


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Results for ^VIX:


,^VIX Close,^VIX daily move%,^VIX trend signal,^VIX cumulative change since signal,^VIX Rolling Max Drawdown,^VIX Sortino Ratio,^VIX Calmar Ratio,^VIX Sharpe Ratio
Date,,,,,,,,
2024-10-21,18.370001,1.885747,-1,-12.231244,-0.523723,NaN,-0.449523,-0.021532
2024-10-22,18.200001,-0.925422,-1,-13.043476,-0.528131,-0.021366,-0.258070,-0.011438
2024-10-23,19.240000,5.714280,1,5.714280,-0.501167,NaN,-0.729719,-0.041877
2024-10-24,19.080000,-0.831600,1,4.835160,-0.505315,0.033461,0.513930,0.019068
2024-10-25,20.330000,6.551363,1,11.703292,-0.472906,NaN,-1.222882,-0.169247
2024-10-28,19.799999,-2.606988,1,8.791200,-0.486648,-0.140130,-1.053010,-0.126140
2024-10-29,19.340000,-2.323228,1,6.263733,-0.498574,-0.132152,-1.029798,-0.121151
2024-10-30,20.350000,5.222338,1,11.813184,-0.472388,NaN,-0.388920,-0.029010
2024-10-31,23.160000,13.808351,1,27.252741,-0.399533,NaN,17.931816,0.185701


Results for ^TNX:


,^TNX Close,^TNX daily move%,^TNX trend signal,^TNX cumulative change since signal,^TNX Rolling Max Drawdown,^TNX Sortino Ratio,^TNX Calmar Ratio,^TNX Sharpe Ratio
Date,,,,,,,,
2024-10-21,4.182,2.676165,1,2.676165,-0.161588,NaN,23.783950,0.593306
2024-10-22,4.204,0.526060,1,3.216304,-0.157177,NaN,21.013697,0.562457
2024-10-23,4.242,0.903904,1,4.149279,-0.149559,NaN,17.677393,0.530101
2024-10-24,4.200,-0.990106,1,3.118091,-0.157979,0.430758,6.064927,0.339713
2024-10-25,4.232,0.761906,1,3.903755,-0.151564,NaN,5.443832,0.320709
2024-10-28,4.278,1.086957,1,5.033143,-0.142342,NaN,7.061155,0.373575
2024-10-29,4.274,-0.093495,1,4.934943,-0.143143,0.405279,5.405670,0.314846
2024-10-30,4.266,-0.187187,1,4.738518,-0.144747,0.334471,4.055232,0.257888
2024-10-31,4.284,0.421944,1,5.180456,-0.141139,NaN,5.541829,0.326500


...

,GOVT Close,GOVT daily move%,GOVT trend signal,GOVT cumulative change since signal,GOVT Rolling Max Drawdown,GOVT Sortino Ratio,GOVT Calmar Ratio,GOVT Sharpe Ratio
Date,,,,,,,,
2024-10-21,22.910000,-0.564241,-1,-0.564241,-0.125906,-0.489956,-1.943077,-0.528487
2024-10-22,22.900000,-0.043650,-1,-0.607644,-0.126288,-0.445150,-1.797119,-0.479551
2024-10-23,22.860001,-0.174668,-1,-0.781251,-0.127814,-0.397265,-1.647026,-0.441808
2024-10-24,22.900000,0.174974,-1,-0.607644,-0.126288,NaN,-0.932101,-0.268558
2024-10-25,22.860001,-0.174668,-1,-0.781251,-0.127814,-0.184572,-0.821265,-0.239943
2024-10-28,22.830000,-0.131237,-1,-0.911463,-0.128958,-0.239074,-1.044936,-0.318970
2024-10-29,22.840000,0.043803,-1,-0.868059,-0.128577,NaN,-0.745667,-0.226939
2024-10-30,22.820000,-0.087568,-1,-0.954866,-0.129340,-0.178127,-0.784770,-0.239614
2024-10-31,22.820000,0.000000,-1,-0.954866,-0.129340,NaN,-0.822981,-0.253352


Results for AAPL:


,AAPL Close,AAPL daily move%,AAPL trend signal,AAPL cumulative change since signal,AAPL Rolling Max Drawdown,AAPL Sortino Ratio,AAPL Calmar Ratio,AAPL Sharpe Ratio
Date,,,,,,,,
2024-10-21,236.479996,0.629785,1,3.023439,0.000000,NaN,4.137184,0.279395
2024-10-22,235.860001,-0.262177,1,2.753336,-0.002622,0.217795,3.482893,0.245462
2024-10-23,230.759995,-2.162302,1,0.531498,-0.024188,0.117473,1.726271,0.126018
2024-10-24,230.570007,-0.082331,-1,-0.082331,-0.024991,0.088487,1.227544,0.094836
2024-10-25,231.410004,0.364313,-1,0.281682,-0.021439,NaN,3.936604,0.275149
2024-10-28,233.399994,0.859941,-1,1.144046,-0.013024,NaN,2.770973,0.228308
2024-10-29,233.669998,0.115683,1,0.115683,-0.011883,NaN,1.295219,0.134335
2024-10-30,230.100006,-1.527792,1,-1.413877,-0.026979,0.025902,0.333548,0.035540
2024-10-31,225.910004,-1.820948,-1,-1.820948,-0.044697,-0.032992,-0.354125,-0.038691


Results for SFM:


,SFM Close,SFM daily move%,SFM trend signal,SFM cumulative change since signal,SFM Rolling Max Drawdown,SFM Sortino Ratio,SFM Calmar Ratio,SFM Sharpe Ratio
Date,,,,,,,,
2024-10-21,117.419998,1.398959,-1,1.398959,0.000000,NaN,5.346115,0.343769
2024-10-22,118.269997,0.723896,1,0.723896,0.000000,NaN,6.932574,0.406640
2024-10-23,118.849998,0.490405,1,1.217851,0.000000,NaN,7.245904,0.417505
2024-10-24,118.779999,-0.058898,1,1.158236,-0.000589,0.362324,4.521717,0.324528
2024-10-25,119.320000,0.454623,1,1.618124,0.000000,NaN,4.940532,0.344105
2024-10-28,120.180000,0.720751,1,2.350538,0.000000,NaN,3.546495,0.293781
2024-10-29,119.690002,-0.407720,1,1.933235,-0.004077,0.420310,4.456227,0.363203
2024-10-30,118.839996,-0.710173,1,1.209332,-0.011150,0.428956,4.528765,0.368743
2024-10-31,128.429993,8.069671,1,9.376592,0.000000,NaN,16.177376,0.334452


In [4]:
import yfinance as yf
import pandas as pd

# List of tickers to analyze
tickers = ['^VIX', '^TNX', '^IRX',
           'VXX', 'UVXY',
           'MTUM', 
           'SPY', 'QQQ', 'QQQM', 'VOOG', 'VGT', 'XLU',
           'GLD', 'IAU', 'PHYS',
           'TLT', 'TMF', 'SHY', 'EDV', 'ZROZ', 'GOVT',
           'AAPL', 'SFM']

# Initialize a dictionary to store results for each ticker
results = {}

# Define a target return (for instance, the risk-free rate)
target_return = 0.0  # Adjust this as needed
rolling_window = 20   # Define the window size for rolling calculations

# ** Data Processing Loop for Each Ticker **
for ticker in tickers:
    # ** 1. Data Downloading Section **
    try:
        data = yf.download(ticker, start="2022-01-01", end="2024-12-31", interval="1d")

        # Check if the data is empty
        if data.empty:
            print(f"No data found for {ticker}. Skipping.")
            continue

        # ** 2. Moving Averages Section **
        data['3_SMA'] = data['Close'].rolling(window=3).mean()
        data['5_SMA'] = data['Close'].rolling(window=5).mean()
        data['20_SMA'] = data['Close'].rolling(window=20).mean()
        data['50_SMA'] = data['Close'].rolling(window=50).mean()

        # ** 3. Market Trend Section **
        data['Market_Trend'] = 0
        data.loc[data['3_SMA'] > data['5_SMA'], 'Market_Trend'] = 1
        data.loc[data['3_SMA'] < data['5_SMA'], 'Market_Trend'] = -1

        # ** 4. Price Change Calculations Section **
        data['Price_Change%'] = data['Close'].pct_change() * 100 
        data['Cumulative_Returns'] = (1 + data['Price_Change%'] / 100).cumprod()

        # ** 5. Rolling Maximum Drawdown Calculation Section **
        data['Rolling_Peak'] = data['Cumulative_Returns'].cummax()
        data['Rolling_Drawdown'] = (data['Cumulative_Returns'] - data['Rolling_Peak']) / data['Rolling_Peak']

        # ** 6. Cumulative Change Calculation Section using vectorized operations **
        current_trend_shifted = data['Market_Trend'].shift(1)
        current_base_price = data['Close'].shift(1).where(current_trend_shifted != data['Market_Trend']).ffill()
        data['Cumulative_Change%'] = ((data['Close'] - current_base_price) / current_base_price) * 100

        # ** 7. Rolling Up/Down Ratio Calculation **
        data['Up_Day'] = data['Price_Change%'].apply(lambda x: x if x > 0 else 0)
        data['Down_Day'] = data['Price_Change%'].apply(lambda x: -x if x < 0 else 0)
        data['Rolling_Up_Down_Ratio'] = data['Up_Day'].rolling(window=rolling_window).mean() / data['Down_Day'].rolling(window=rolling_window).mean()
        
        # ** 8. Rolling True Strength Index (TSI) Calculation **
        # Calculate the TSI using the standard approach: smooth the price change, then smooth those
        data['Smooth_Price_Change'] = data['Price_Change%'].ewm(span=25, adjust=False).mean()  # Fast TSI
        data['Smooth_Price_Change_Slow'] = data['Price_Change%'].ewm(span=13, adjust=False).mean()  # Slow TSI
        data['TSI'] = 100 * (data['Smooth_Price_Change'] - data['Smooth_Price_Change_Slow']) / data['Smooth_Price_Change_Slow']
        data['Rolling_TSI'] = data['TSI'].rolling(window=rolling_window).mean()

        # ** 9. Rolling Volatility Calculation **
        data['Rolling_Volatility'] = data['Price_Change%'].rolling(window=rolling_window).std()

        # ** 10a. Rolling Maximum Consecutive Up Days Calculation **
        data['Consecutive_Up_Days'] = (data['Price_Change%'] > 0).astype(int)
        data['Rolling_Consecutive_Up_Days'] = data['Consecutive_Up_Days'].groupby((data['Consecutive_Up_Days'] != data['Consecutive_Up_Days'].shift()).cumsum()).cumsum()
        
        # ** 10b. Max Rolling Consecutive Up Days in a 20-day Window **
        # Create a temporary DataFrame to hold the consecutive up days counts
        consecutive_counts = data['Consecutive_Up_Days'].rolling(window=rolling_window).apply(
            lambda x: (x != 0).astype(int).groupby((x != 0).ne((x.shift())).cumsum()).cumsum().max(),
            raw=False
        )

        # Assign the calculated max consecutive up days to the new column
        data['Max_Rolling_Consecutive_Up_Days'] = consecutive_counts

        # ** 11. Preparing the Trend Indicator DataFrame **
        trend_indicator = data[['Close', 'Price_Change%', 'Market_Trend', 'Cumulative_Change%', 
                                'Rolling_Drawdown', 'Rolling_Up_Down_Ratio', 'Rolling_TSI', 
                                'Rolling_Volatility', 'Rolling_Consecutive_Up_Days', 'Max_Rolling_Consecutive_Up_Days']].dropna()
        trend_indicator.columns = [f'{ticker} Close',
                                   f'{ticker} daily move%',
                                   f'{ticker} trend signal',
                                   f'{ticker} cumulative change since signal',
                                   f'{ticker} Rolling Max Drawdown',
                                   f'{ticker} Rolling Up Down Ratio',
                                   f'{ticker} Rolling TSI',
                                   f'{ticker} Rolling Volatility',
                                   f'{ticker} Consecutive Up Days',
                                   f'{ticker} Max Rolling Consecutive Up Days']

        # ** 12. Sortino Ratio Calculation Section **
        average_return = data['Price_Change%'].rolling(window=rolling_window).mean()
        downside_returns = data['Price_Change%'][data['Price_Change%'] < target_return].rolling(window=rolling_window)
        downside_deviation = downside_returns.apply(lambda x: (x ** 2).mean() ** 0.5 if not x.empty else 0)

        # Calculate Sortino Ratio
        data['Sortino Ratio'] = (average_return - target_return) / downside_deviation
        data['Sortino Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities

        # ** 13. Calmar Ratio Calculation Section **
        annualized_return = ((1 + average_return / 100) ** 252) - 1
        data['Calmar Ratio'] = annualized_return / abs(data['Rolling_Drawdown'].min())  # Use current rolling drawdown

        # ** 14. Sharpe Ratio Calculation Section **
        trend_indicator[f'{ticker} Sortino Ratio'] = data['Sortino Ratio']
        trend_indicator[f'{ticker} Calmar Ratio'] = data['Calmar Ratio']

        results[ticker] = trend_indicator  # Store the results for this ticker

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# ** Risk-Free Rate Calculation Section **
if "^TNX" in results:  # Ensure ^TNX has been processed successfully
    treasury_yield = results["^TNX"]['^TNX Close']
    risk_free_rate = treasury_yield / 100 / 252  # Convert from annual to daily
else:
    print("Warning: ^TNX results not found. Defaulting risk-free rate to 0.")
    risk_free_rate = 0.0  # Fallback risk-free rate

# ** Calculate Sharpe Ratio for all tickers using the risk-free rate **
for ticker in tickers:
    if ticker in results:
        data = results[ticker]
        rolling_average_return = data[f'{ticker} daily move%'].rolling(window=rolling_window).mean()
        rolling_std_dev_returns = data[f'{ticker} daily move%'].rolling(window=rolling_window).std()

        # Define Sharpe Ratio; replace infinities for stability
        data[f'{ticker} Sharpe Ratio'] = (rolling_average_return - risk_free_rate) / rolling_std_dev_returns
        data[f'{ticker} Sharpe Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities
        results[ticker] = data  # Update results with Sharpe Ratio

# ** 15. Display Results Section **
for ticker in tickers:
    if ticker in results:  # Only display results for tickers that were processed successfully
        print(f"Results for {ticker}:")
        display(results[ticker].tail(10))  # Display the last 10 entries


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Results for ^VIX:


,^VIX Close,^VIX daily move%,^VIX trend signal,^VIX cumulative change since signal,^VIX Rolling Max Drawdown,^VIX Rolling Up Down Ratio,^VIX Rolling TSI,^VIX Rolling Volatility,^VIX Consecutive Up Days,^VIX Max Rolling Consecutive Up Days,^VIX Sortino Ratio,^VIX Calmar Ratio,^VIX Sharpe Ratio
Date,,,,,,,,,,,,,
2024-10-23,19.240000,5.714280,1,5.714280,-0.501167,1.704468,-13.716312,6.931931,1,1.0,NaN,40.088578,0.192090
2024-10-24,19.080000,-0.831600,1,4.835160,-0.505315,1.679064,-1.171510,6.940018,0,1.0,0.306563,37.234029,0.187746
2024-10-25,20.330000,6.551363,1,11.703292,-0.472906,1.580224,-0.502782,6.728515,1,1.0,NaN,22.660631,0.165457
2024-10-28,19.799999,-2.606988,1,8.791200,-0.486648,1.530348,0.851402,6.758425,0,1.0,0.264761,19.175040,0.155471
2024-10-29,19.340000,-2.323228,1,6.263733,-0.498574,1.085150,12.974732,5.920554,0,1.0,0.045501,0.841765,0.030141
2024-10-30,20.350000,5.222338,1,11.813184,-0.472388,1.266032,13.922333,6.003234,1,1.0,NaN,4.179157,0.088790
2024-10-31,23.160000,13.808351,1,27.252741,-0.399533,1.400638,13.990984,6.478405,2,2.0,NaN,9.640785,0.123921
2024-11-01,21.879999,-5.526773,1,20.219771,-0.432720,1.426263,14.053287,6.439039,0,2.0,0.206596,10.688340,0.130270
2024-11-04,21.980000,0.457040,1,20.769223,-0.430127,0.984287,14.906675,5.043076,1,2.0,NaN,-0.111151,-0.006166


Results for ^TNX:


,^TNX Close,^TNX daily move%,^TNX trend signal,^TNX cumulative change since signal,^TNX Rolling Max Drawdown,^TNX Rolling Up Down Ratio,^TNX Rolling TSI,^TNX Rolling Volatility,^TNX Consecutive Up Days,^TNX Max Rolling Consecutive Up Days,^TNX Sortino Ratio,^TNX Calmar Ratio,^TNX Sharpe Ratio
Date,,,,,,,,,,,,,
2024-10-23,4.242,0.903904,1,4.149279,-0.149559,3.020263,-17.941657,1.314763,3,7.0,NaN,12.220964,0.444849
2024-10-24,4.200,-0.990106,1,3.118091,-0.157979,2.540324,-14.480620,1.360013,0,7.0,0.477058,9.912329,0.383925
2024-10-25,4.232,0.761906,1,3.903755,-0.151564,3.170624,-7.265197,1.305215,1,7.0,NaN,13.492125,0.471671
2024-10-28,4.278,1.086957,1,5.033143,-0.142342,3.113036,-4.669204,1.296719,2,7.0,NaN,12.804836,0.462160
2024-10-29,4.274,-0.093495,1,4.934943,-0.143143,4.189937,-23.262449,1.207302,0,7.0,0.635660,16.100823,0.556785
2024-10-30,4.266,-0.187187,1,4.738518,-0.144747,3.756941,-19.618768,1.217091,0,6.0,0.587898,13.114847,0.498520
2024-10-31,4.284,0.421944,1,5.180456,-0.141139,3.462729,-16.844282,1.189034,1,5.0,NaN,10.603659,0.455812
2024-11-01,4.361,1.797389,1,7.070958,-0.125702,3.098141,-15.206909,1.029227,2,4.0,NaN,8.005582,0.448601
2024-11-04,4.309,-1.192388,1,5.794256,-0.136127,2.235873,-12.438354,1.079641,0,3.0,0.339783,5.058743,0.320085


...

,GOVT Close,GOVT daily move%,GOVT trend signal,GOVT cumulative change since signal,GOVT Rolling Max Drawdown,GOVT Rolling Up Down Ratio,GOVT Rolling TSI,GOVT Rolling Volatility,GOVT Consecutive Up Days,GOVT Max Rolling Consecutive Up Days,GOVT Sortino Ratio,GOVT Calmar Ratio,GOVT Sharpe Ratio
Date,,,,,,,,,,,,,
2024-10-23,22.860001,-0.174668,-1,-0.781251,-0.127814,0.298520,-27.058595,0.284085,0,2.0,-0.418508,-1.557394,-0.447444
2024-10-24,22.900000,0.174974,-1,-0.607644,-0.126288,0.351000,-22.195743,0.291554,1,2.0,NaN,-1.442494,-0.398657
2024-10-25,22.860001,-0.174668,-1,-0.781251,-0.127814,0.255050,-31.306271,0.274852,0,2.0,-0.460435,-1.688437,-0.508988
2024-10-28,22.830000,-0.131237,-1,-0.911463,-0.128958,0.263767,-26.450177,0.273505,0,2.0,-0.441373,-1.625424,-0.488837
2024-10-29,22.840000,0.043803,-1,-0.868059,-0.128577,0.275843,-21.060222,0.274803,1,2.0,NaN,-1.602924,-0.478558
2024-10-30,22.820000,-0.087568,-1,-0.954866,-0.129340,0.289264,-17.668985,0.273362,0,2.0,-0.422498,-1.515313,-0.450296
2024-10-31,22.820000,0.000000,-1,-0.954866,-0.129340,0.325468,-14.570960,0.267439,0,2.0,NaN,-1.307912,-0.388333
2024-11-01,22.670000,-0.657316,-1,-1.605906,-0.135063,0.333307,-13.242511,0.258885,0,2.0,-0.307128,-1.267811,-0.387213
2024-11-04,22.770000,0.441113,-1,-1.171877,-0.131248,0.534089,-10.461123,0.280750,1,2.0,NaN,-0.833433,-0.224591


Results for AAPL:


,AAPL Close,AAPL daily move%,AAPL trend signal,AAPL cumulative change since signal,AAPL Rolling Max Drawdown,AAPL Rolling Up Down Ratio,AAPL Rolling TSI,AAPL Rolling Volatility,AAPL Consecutive Up Days,AAPL Max Rolling Consecutive Up Days,AAPL Sortino Ratio,AAPL Calmar Ratio,AAPL Sharpe Ratio
Date,,,,,,,,,,,,,
2024-10-23,230.759995,-2.162302,1,0.531498,-0.024188,1.214508,20.778625,1.403716,0,3.0,0.076111,1.000079,0.075025
2024-10-24,230.570007,-0.082331,-1,-0.082331,-0.024991,1.153198,12.772771,1.401010,0,3.0,0.055301,0.693070,0.054102
2024-10-25,231.410004,0.364313,-1,0.281682,-0.021439,1.177968,67.316203,1.402480,1,3.0,NaN,0.818062,0.062802
2024-10-28,233.399994,0.859941,-1,1.144046,-0.013024,1.034049,69.351280,1.318498,2,3.0,NaN,0.142828,0.012676
2024-10-29,233.669998,0.115683,1,0.115683,-0.011883,1.480873,22.566690,1.123671,3,3.0,NaN,1.735165,0.149693
2024-10-30,230.100006,-1.527792,1,-1.413877,-0.026979,1.186121,13.153267,1.185476,0,3.0,0.056406,0.727522,0.066824
2024-10-31,225.910004,-1.820948,-1,-1.820948,-0.044697,1.025984,19.747082,1.254481,0,3.0,0.008757,0.107837,0.010078
2024-11-01,222.910004,-1.327962,-1,-3.124729,-0.057383,0.859480,13.865114,1.283358,0,3.0,-0.057667,-0.590888,-0.061397
2024-11-04,222.009995,-0.403754,-1,-3.515868,-0.061189,1.029643,14.455711,1.180980,0,3.0,0.010235,0.116673,0.011578


Results for SFM:


,SFM Close,SFM daily move%,SFM trend signal,SFM cumulative change since signal,SFM Rolling Max Drawdown,SFM Rolling Up Down Ratio,SFM Rolling TSI,SFM Rolling Volatility,SFM Consecutive Up Days,SFM Max Rolling Consecutive Up Days,SFM Sortino Ratio,SFM Calmar Ratio,SFM Sharpe Ratio
Date,,,,,,,,,,,,,
2024-10-23,118.849998,0.490405,1,1.217851,0.000000,2.327346,4.790009,1.191591,3,4.0,NaN,4.881198,0.325451
2024-10-24,118.779999,-0.058898,1,1.158236,-0.000589,3.100225,4.936142,1.110840,0,4.0,0.338994,6.452895,0.414707
2024-10-25,119.320000,0.454623,1,1.618124,0.000000,3.648350,3.720065,1.085922,1,4.0,NaN,7.695119,0.469775
2024-10-28,120.180000,0.720751,1,2.350538,0.000000,3.229289,4.253281,0.999660,2,4.0,NaN,5.742987,0.429535
2024-10-29,119.690002,-0.407720,1,1.933235,-0.004077,2.801307,5.616966,1.016708,0,4.0,0.281706,4.799823,0.377338
2024-10-30,118.839996,-0.710173,1,1.209332,-0.011150,2.529534,9.114619,1.036743,0,4.0,0.265211,4.366821,0.347966
2024-10-31,128.429993,8.069671,1,9.376592,0.000000,4.172447,7.314063,2.011008,1,3.0,NaN,16.385267,0.372164
2024-11-01,130.210007,1.385980,1,10.892530,0.000000,4.111190,6.386353,2.005028,2,3.0,NaN,15.698552,0.366063
2024-11-04,131.789993,1.213414,1,12.238116,0.000000,4.336399,5.320638,2.002831,3,3.0,NaN,18.349668,0.392999


In [5]:
import yfinance as yf
import pandas as pd

# List of tickers to analyze
tickers = ['^VIX', '^TNX', '^IRX',
           'VXX', 'UVXY',
           'MTUM', 
           'SPY', 'QQQ', 'QQQM', 'VOOG', 'VGT', 'XLU',
           'GLD', 'IAU', 'PHYS',
           'TLT', 'TMF', 'SHY', 'EDV', 'ZROZ', 'GOVT',
           'AAPL', 'SFM']

# Initialize a dictionary to store results for each ticker
results = {}

# Define a target return (for instance, the risk-free rate)
target_return = 0.0  # Adjust this as needed
rolling_window = 20   # Define the window size for rolling calculations

# ** Data Processing Loop for Each Ticker **
for ticker in tickers:
    # ** 1. Data Downloading Section **
    try:
        data = yf.download(ticker, start="2022-01-01", end="2024-12-31", interval="1d")

        # Check if the data is empty
        if data.empty:
            print(f"No data found for {ticker}. Skipping.")
            continue

        # ** 2. Moving Averages Section **
        data['3_SMA'] = data['Close'].rolling(window=3).mean()
        data['5_SMA'] = data['Close'].rolling(window=5).mean()
        data['20_SMA'] = data['Close'].rolling(window=20).mean()
        data['50_SMA'] = data['Close'].rolling(window=50).mean()

        # ** 3. Market Trend Section **
        data['Market_Trend'] = 0
        data.loc[data['3_SMA'] > data['5_SMA'], 'Market_Trend'] = 1
        data.loc[data['3_SMA'] < data['5_SMA'], 'Market_Trend'] = -1

        # ** 4. Price Change Calculations Section **
        data['Price_Change%'] = data['Close'].pct_change() * 100 
        data['Cumulative_Returns'] = (1 + data['Price_Change%'] / 100).cumprod()

        # ** 5. Rolling Maximum Drawdown Calculation Section **
        data['Rolling_Peak'] = data['Cumulative_Returns'].cummax()
        data['Rolling_Drawdown'] = (data['Cumulative_Returns'] - data['Rolling_Peak']) / data['Rolling_Peak']

        # ** 6. Cumulative Change Calculation Section using vectorized operations **
        current_trend_shifted = data['Market_Trend'].shift(1)
        current_base_price = data['Close'].shift(1).where(current_trend_shifted != data['Market_Trend']).ffill()
        data['Cumulative_Change%'] = ((data['Close'] - current_base_price) / current_base_price) * 100

        # ** 7. Rolling Up/Down Ratio Calculation **
        data['Up_Day'] = data['Price_Change%'].apply(lambda x: x if x > 0 else 0)
        data['Down_Day'] = data['Price_Change%'].apply(lambda x: -x if x < 0 else 0)
        data['Rolling_Up_Down_Ratio'] = data['Up_Day'].rolling(window=rolling_window).mean() / data['Down_Day'].rolling(window=rolling_window).mean()
        
        # ** 8. Rolling True Strength Index (TSI) Calculation **
        data['Smooth_Price_Change'] = data['Price_Change%'].ewm(span=25, adjust=False).mean()  # Fast TSI
        data['Smooth_Price_Change_Slow'] = data['Price_Change%'].ewm(span=13, adjust=False).mean()  # Slow TSI
        data['TSI'] = 100 * (data['Smooth_Price_Change'] - data['Smooth_Price_Change_Slow']) / data['Smooth_Price_Change_Slow']
        data['Rolling_TSI'] = data['TSI'].rolling(window=rolling_window).mean()

        # ** 9. Rolling Volatility Calculation **
        data['Rolling_Volatility'] = data['Price_Change%'].rolling(window=rolling_window).std()

        # ** 10a. Rolling Maximum Consecutive Up Days Calculation **
        data['Consecutive_Up_Days'] = (data['Price_Change%'] > 0).astype(int)
        data['Rolling_Consecutive_Up_Days'] = data['Consecutive_Up_Days'].groupby((data['Consecutive_Up_Days'] != data['Consecutive_Up_Days'].shift()).cumsum()).cumsum()
        
        # ** 10b. Max Rolling Consecutive Up Days in a 20-day Window **
        consecutive_counts = data['Consecutive_Up_Days'].rolling(window=rolling_window).apply(
            lambda x: (x != 0).astype(int).groupby((x != 0).ne((x.shift())).cumsum()).cumsum().max(),
            raw=False
        )
        data['Max_Rolling_Consecutive_Up_Days'] = consecutive_counts

        # ** 11. RSI Calculation (Relative Strength Index) **
        delta = data['Close'].diff(1)
        gain = (delta.where(delta > 0, 0)).rolling(window=rolling_window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=rolling_window).mean()
        rs = gain / loss
        data['RSI'] = 100 - (100 / (1 + rs))

        # ** 12. Momentum Calculation **
        data['Momentum'] = data['Close'].diff(rolling_window)

        # ** 13. ATR Calculation (Average True Range) **
        data['High_Low'] = data['High'] - data['Low']
        data['High_Close'] = (data['High'] - data['Close'].shift(1)).abs()
        data['Low_Close'] = (data['Low'] - data['Close'].shift(1)).abs()
        data['True_Range'] = data[['High_Low', 'High_Close', 'Low_Close']].max(axis=1)
        data['ATR'] = data['True_Range'].rolling(window=rolling_window).mean()

        # ** 14. Preparing the Trend Indicator DataFrame **
        trend_indicator = data[['Close', 'Price_Change%', 'Market_Trend', 'Cumulative_Change%', 
                                'Rolling_Drawdown', 'Rolling_Up_Down_Ratio', 'Rolling_TSI', 
                                'Rolling_Volatility', 'Rolling_Consecutive_Up_Days', 
                                'Max_Rolling_Consecutive_Up_Days', 'RSI', 'Momentum', 'ATR']].dropna()
        trend_indicator.columns = [f'{ticker} Close',
                                   f'{ticker} daily move%',
                                   f'{ticker} trend signal',
                                   f'{ticker} cumulative change since signal',
                                   f'{ticker} Rolling Max Drawdown',
                                   f'{ticker} Rolling Up Down Ratio',
                                   f'{ticker} Rolling TSI',
                                   f'{ticker} Rolling Volatility',
                                   f'{ticker} Consecutive Up Days',
                                   f'{ticker} Max Rolling Consecutive Up Days',
                                   f'{ticker} RSI',
                                   f'{ticker} Momentum',
                                   f'{ticker} ATR']

        # ** 15. Sortino Ratio Calculation Section **
        average_return = data['Price_Change%'].rolling(window=rolling_window).mean()
        downside_returns = data['Price_Change%'][data['Price_Change%'] < target_return].rolling(window=rolling_window)
        downside_deviation = downside_returns.apply(lambda x: (x ** 2).mean() ** 0.5 if not x.empty else 0)

        # Calculate Sortino Ratio
        data['Sortino Ratio'] = (average_return - target_return) / downside_deviation
        data['Sortino Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities

        # ** 16. Calmar Ratio Calculation Section **
        annualized_return = ((1 + average_return / 100) ** 252) - 1
        data['Calmar Ratio'] = annualized_return / abs(data['Rolling_Drawdown'].min())  # Use current rolling drawdown

        # ** 17. Sharpe Ratio Calculation Section **
        trend_indicator[f'{ticker} Sortino Ratio'] = data['Sortino Ratio']
        trend_indicator[f'{ticker} Calmar Ratio'] = data['Calmar Ratio']

        results[ticker] = trend_indicator  # Store the results for this ticker

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# ** Risk-Free Rate Calculation Section **
if "^TNX" in results:  # Ensure ^TNX has been processed successfully
    treasury_yield = results["^TNX"]['^TNX Close']
    risk_free_rate = treasury_yield / 100 / 252  # Convert from annual to daily
else:
    print("Warning: ^TNX results not found. Defaulting risk-free rate to 0.")
    risk_free_rate = 0.0  # Fallback risk-free rate

# ** Calculate Sharpe Ratio for all tickers using the risk-free rate **
for ticker in tickers:
    if ticker in results:
        data = results[ticker]
        rolling_average_return = data[f'{ticker} daily move%'].rolling(window=rolling_window).mean()
        rolling_std_dev_returns = data[f'{ticker} daily move%'].rolling(window=rolling_window).std()

        # Define Sharpe Ratio; replace infinities for stability
        data[f'{ticker} Sharpe Ratio'] = (rolling_average_return - risk_free_rate) / rolling_std_dev_returns
        data[f'{ticker} Sharpe Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities
        results[ticker] = data  # Update results with Sharpe Ratio

# ** 18. Display Results Section **
for ticker in tickers:
    if ticker in results:  # Only display results for tickers that were processed successfully
        print(f"Results for {ticker}:")
        display(results[ticker].tail(10))  # Display the last 10 entries


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Results for ^VIX:


,^VIX Close,^VIX daily move%,^VIX trend signal,^VIX cumulative change since signal,^VIX Rolling Max Drawdown,^VIX Rolling Up Down Ratio,^VIX Rolling TSI,^VIX Rolling Volatility,^VIX Consecutive Up Days,^VIX Max Rolling Consecutive Up Days,^VIX RSI,^VIX Momentum,^VIX ATR,^VIX Sortino Ratio,^VIX Calmar Ratio,^VIX Sharpe Ratio
Date,,,,,,,,,,,,,,,,
2024-10-23,19.240000,5.714280,1,5.714280,-0.501167,1.704468,-13.716312,6.931931,1,1.0,59.958400,3.830000,1.6970,NaN,40.088578,0.192090
2024-10-24,19.080000,-0.831600,1,4.835160,-0.505315,1.679064,-1.171510,6.940018,0,1.0,59.586566,3.710000,1.7310,0.306563,37.234029,0.187746
2024-10-25,20.330000,6.551363,1,11.703292,-0.472906,1.580224,-0.502782,6.728515,1,1.0,58.863760,3.370001,1.7565,NaN,22.660631,0.165457
2024-10-28,19.799999,-2.606988,1,8.791200,-0.486648,1.530348,0.851402,6.758425,0,1.0,57.949249,3.070000,1.7615,0.264761,19.175040,0.155471
2024-10-29,19.340000,-2.323228,1,6.263733,-0.498574,1.085150,12.974732,5.920554,0,1.0,50.232018,0.080000,1.6290,0.045501,0.841765,0.030141
2024-10-30,20.350000,5.222338,1,11.813184,-0.472388,1.266032,13.922333,6.003234,1,1.0,54.052546,1.450001,1.5970,NaN,4.179157,0.088790
2024-10-31,23.160000,13.808351,1,27.252741,-0.399533,1.400638,13.990984,6.478405,2,2.0,56.985873,2.670000,1.6580,NaN,9.640785,0.123921
2024-11-01,21.879999,-5.526773,1,20.219771,-0.432720,1.426263,14.053287,6.439039,0,2.0,56.985873,2.670000,1.6575,0.206596,10.688340,0.130270
2024-11-04,21.980000,0.457040,1,20.769223,-0.430127,0.984287,14.906675,5.043076,1,2.0,47.908745,-0.660000,1.5335,NaN,-0.111151,-0.006166


Results for ^TNX:


,^TNX Close,^TNX daily move%,^TNX trend signal,^TNX cumulative change since signal,^TNX Rolling Max Drawdown,^TNX Rolling Up Down Ratio,^TNX Rolling TSI,^TNX Rolling Volatility,^TNX Consecutive Up Days,^TNX Max Rolling Consecutive Up Days,^TNX RSI,^TNX Momentum,^TNX ATR,^TNX Sortino Ratio,^TNX Calmar Ratio,^TNX Sharpe Ratio
Date,,,,,,,,,,,,,,,,
2024-10-23,4.242,0.903904,1,4.149279,-0.149559,3.020263,-17.941657,1.314763,3,7.0,75.081585,0.461,0.06390,NaN,12.220964,0.444849
2024-10-24,4.200,-0.990106,1,3.118091,-0.157979,2.540324,-14.480620,1.360013,0,7.0,71.503639,0.409,0.06360,0.477058,9.912329,0.383925
2024-10-25,4.232,0.761906,1,3.903755,-0.151564,3.170624,-7.265197,1.305215,1,7.0,75.664125,0.483,0.06390,NaN,13.492125,0.471671
2024-10-28,4.278,1.086957,1,5.033143,-0.142342,3.113036,-4.669204,1.296719,2,7.0,75.481737,0.476,0.06455,NaN,12.804836,0.462160
2024-10-29,4.274,-0.093495,1,4.934943,-0.143143,4.189937,-23.262449,1.207302,0,7.0,80.204730,0.531,0.06260,0.635660,16.100823,0.556785
2024-10-30,4.266,-0.187187,1,4.738518,-0.144747,3.756941,-19.618768,1.217091,0,6.0,78.461456,0.481,0.06290,0.587898,13.114847,0.498520
2024-10-31,4.284,0.421944,1,5.180456,-0.141139,3.462729,-16.844282,1.189034,1,5.0,77.192907,0.434,0.06330,NaN,10.603659,0.455812
2024-11-01,4.361,1.797389,1,7.070958,-0.125702,3.098141,-15.206909,1.029227,2,4.0,75.537562,0.380,0.06385,NaN,8.005582,0.448601
2024-11-04,4.309,-1.192388,1,5.794256,-0.136127,2.235873,-12.438354,1.079641,0,3.0,68.841486,0.283,0.06615,0.339783,5.058743,0.320085


...

,GOVT Close,GOVT daily move%,GOVT trend signal,GOVT cumulative change since signal,GOVT Rolling Max Drawdown,GOVT Rolling Up Down Ratio,GOVT Rolling TSI,GOVT Rolling Volatility,GOVT Consecutive Up Days,GOVT Max Rolling Consecutive Up Days,GOVT RSI,GOVT Momentum,GOVT ATR,GOVT Sortino Ratio,GOVT Calmar Ratio,GOVT Sharpe Ratio
Date,,,,,,,,,,,,,,,,
2024-10-23,22.860001,-0.174668,-1,-0.781251,-0.127814,0.298520,-27.058595,0.284085,0,2.0,22.935871,-0.590000,0.0880,-0.418508,-1.557394,-0.447444
2024-10-24,22.900000,0.174974,-1,-0.607644,-0.126288,0.351000,-22.195743,0.291554,1,2.0,25.892875,-0.540001,0.0885,NaN,-1.442494,-0.398657
2024-10-25,22.860001,-0.174668,-1,-0.781251,-0.127814,0.255050,-31.306271,0.274852,0,2.0,20.183560,-0.650000,0.0895,-0.460435,-1.688437,-0.508988
2024-10-28,22.830000,-0.131237,-1,-0.911463,-0.128958,0.263767,-26.450177,0.273505,0,2.0,20.754768,-0.620001,0.0895,-0.441373,-1.625424,-0.488837
2024-10-29,22.840000,0.043803,-1,-0.868059,-0.128577,0.275843,-21.060222,0.274803,1,2.0,21.495392,-0.610001,0.0875,NaN,-1.602924,-0.478558
2024-10-30,22.820000,-0.087568,-1,-0.954866,-0.129340,0.289264,-17.668985,0.273362,0,2.0,22.330182,-0.570000,0.0880,-0.422498,-1.515313,-0.450296
2024-10-31,22.820000,0.000000,-1,-0.954866,-0.129340,0.325468,-14.570960,0.267439,0,2.0,24.468176,-0.480000,0.0885,NaN,-1.307912,-0.388333
2024-11-01,22.670000,-0.657316,-1,-1.605906,-0.135063,0.333307,-13.242511,0.258885,0,2.0,25.000104,-0.459999,0.0875,-0.307128,-1.267811,-0.387213
2024-11-04,22.770000,0.441113,-1,-1.171877,-0.131248,0.534089,-10.461123,0.280750,1,2.0,34.736937,-0.289999,0.0905,NaN,-0.833433,-0.224591


Results for AAPL:


,AAPL Close,AAPL daily move%,AAPL trend signal,AAPL cumulative change since signal,AAPL Rolling Max Drawdown,AAPL Rolling Up Down Ratio,AAPL Rolling TSI,AAPL Rolling Volatility,AAPL Consecutive Up Days,AAPL Max Rolling Consecutive Up Days,AAPL RSI,AAPL Momentum,AAPL ATR,AAPL Sortino Ratio,AAPL Calmar Ratio,AAPL Sharpe Ratio
Date,,,,,,,,,,,,,,,,
2024-10-23,230.759995,-2.162302,1,0.531498,-0.024188,1.214508,20.778625,1.403716,0,3.0,54.392636,4.389999,4.336499,0.076111,1.000079,0.075025
2024-10-24,230.570007,-0.082331,-1,-0.082331,-0.024991,1.153198,12.772771,1.401010,0,3.0,53.111615,3.050003,4.302499,0.055301,0.693070,0.054102
2024-10-25,231.410004,0.364313,-1,0.281682,-0.021439,1.177968,67.316203,1.402480,1,3.0,53.650678,3.620010,4.373999,NaN,0.818062,0.062802
2024-10-28,233.399994,0.859941,-1,1.144046,-0.013024,1.034049,69.351280,1.318498,2,3.0,50.431400,0.399994,4.279498,NaN,0.142828,0.012676
2024-10-29,233.669998,0.115683,1,0.115683,-0.011883,1.480873,22.566690,1.123671,3,3.0,59.362447,7.459991,3.916998,NaN,1.735165,0.149693
2024-10-30,230.100006,-1.527792,1,-1.413877,-0.026979,1.186121,13.153267,1.185476,0,3.0,53.874895,3.320007,3.905499,0.056406,0.727522,0.066824
2024-10-31,225.910004,-1.820948,-1,-1.820948,-0.044697,1.025984,19.747082,1.254481,0,3.0,50.261330,0.240005,3.967500,0.008757,0.107837,0.010078
2024-11-01,222.910004,-1.327962,-1,-3.124729,-0.057383,0.859480,13.865114,1.283358,0,3.0,45.930108,-3.889999,4.056000,-0.057667,-0.590888,-0.061397
2024-11-04,222.009995,-0.403754,-1,-3.515868,-0.061189,1.029643,14.455711,1.180980,0,3.0,50.367132,0.319992,3.942500,0.010235,0.116673,0.011578


Results for SFM:


,SFM Close,SFM daily move%,SFM trend signal,SFM cumulative change since signal,SFM Rolling Max Drawdown,SFM Rolling Up Down Ratio,SFM Rolling TSI,SFM Rolling Volatility,SFM Consecutive Up Days,SFM Max Rolling Consecutive Up Days,SFM RSI,SFM Momentum,SFM ATR,SFM Sortino Ratio,SFM Calmar Ratio,SFM Sharpe Ratio
Date,,,,,,,,,,,,,,,,
2024-10-23,118.849998,0.490405,1,1.217851,0.000000,2.327346,4.790009,1.191591,3,4.0,69.840540,8.709999,2.295500,NaN,4.881198,0.325451
2024-10-24,118.779999,-0.058898,1,1.158236,-0.000589,3.100225,4.936142,1.110840,0,4.0,75.331682,10.309998,2.183000,0.338994,6.452895,0.414707
2024-10-25,119.320000,0.454623,1,1.618124,0.000000,3.648350,3.720065,1.085922,1,4.0,78.138841,11.430000,2.197500,NaN,7.695119,0.469775
2024-10-28,120.180000,0.720751,1,2.350538,0.000000,3.229289,4.253281,0.999660,2,4.0,76.193018,9.769997,2.113000,NaN,5.742987,0.429535
2024-10-29,119.690002,-0.407720,1,1.933235,-0.004077,2.801307,5.616966,1.016708,0,4.0,73.466094,8.720001,2.092001,0.281706,4.799823,0.377338
2024-10-30,118.839996,-0.710173,1,1.209332,-0.011150,2.529534,9.114619,1.036743,0,4.0,71.279347,8.149994,2.131001,0.265211,4.366821,0.347966
2024-10-31,128.429993,8.069671,1,9.376592,0.000000,4.172447,7.314063,2.011008,1,3.0,80.626959,17.389992,2.863001,NaN,16.385267,0.372164
2024-11-01,130.210007,1.385980,1,10.892530,0.000000,4.111190,6.386353,2.005028,2,3.0,80.572223,17.310005,2.925502,NaN,15.698552,0.366063
2024-11-04,131.789993,1.213414,1,12.238116,0.000000,4.336399,5.320638,2.002831,3,3.0,81.493923,18.719994,3.055002,NaN,18.349668,0.392999


In [6]:
import yfinance as yf
import pandas as pd

# List of tickers to analyze including the benchmark
tickers = ['^VIX', '^TNX', '^IRX',
           'VXX', 'UVXY',
           'MTUM', 
           'SPY', 'QQQ', 'QQQM', 'VOOG', 'VGT', 'XLU',
           'GLD', 'IAU', 'PHYS',
           'TLT', 'TMF', 'SHY', 'EDV', 'ZROZ', 'GOVT',
           'AAPL', 'SFM']

# Initialize a dictionary to store results for each ticker
results = {}

# Define a target return (for instance, the risk-free rate)
target_return = 0.0  # Adjust this as needed
rolling_window = 20   # Define the window size for rolling calculations

# Define the benchmark
benchmark_ticker = 'SPY'

# ** Data Processing Loop for Each Ticker **
for ticker in tickers:
    # ** 1. Data Downloading Section **
    try:
        data = yf.download(ticker, start="2022-01-01", end="2024-12-31", interval="1d")

        # Check if the data is empty
        if data.empty:
            print(f"No data found for {ticker}. Skipping.")
            continue
        
        # Add benchmark data for Beta calculation
        if ticker == benchmark_ticker:
            benchmark_data = data['Close']
            continue  # Skip the rest for the benchmark itself

        # ** 2. Moving Averages Section **
        data['3_SMA'] = data['Close'].rolling(window=3).mean()
        data['5_SMA'] = data['Close'].rolling(window=5).mean()
        data['20_SMA'] = data['Close'].rolling(window=20).mean()
        data['50_SMA'] = data['Close'].rolling(window=50).mean()

        # ** 3. Market Trend Section **
        data['Market_Trend'] = 0
        data.loc[data['3_SMA'] > data['5_SMA'], 'Market_Trend'] = 1
        data.loc[data['3_SMA'] < data['5_SMA'], 'Market_Trend'] = -1

        # ** 4. Price Change Calculations Section **
        data['Price_Change%'] = data['Close'].pct_change() * 100 
        data['Cumulative_Returns'] = (1 + data['Price_Change%'] / 100).cumprod()

        # ** 5. Rolling Maximum Drawdown Calculation Section **
        data['Rolling_Peak'] = data['Cumulative_Returns'].cummax()
        data['Rolling_Drawdown'] = (data['Cumulative_Returns'] - data['Rolling_Peak']) / data['Rolling_Peak']

        # ** 6. Cumulative Change Calculation Section using vectorized operations **
        current_trend_shifted = data['Market_Trend'].shift(1)
        current_base_price = data['Close'].shift(1).where(current_trend_shifted != data['Market_Trend']).ffill()
        data['Cumulative_Change%'] = ((data['Close'] - current_base_price) / current_base_price) * 100

        # ** 7. Rolling Up/Down Ratio Calculation **
        data['Up_Day'] = data['Price_Change%'].apply(lambda x: x if x > 0 else 0)
        data['Down_Day'] = data['Price_Change%'].apply(lambda x: -x if x < 0 else 0)
        data['Rolling_Up_Down_Ratio'] = data['Up_Day'].rolling(window=rolling_window).mean() / data['Down_Day'].rolling(window=rolling_window).mean()
        
        # ** 8. Rolling True Strength Index (TSI) Calculation **
        data['Smooth_Price_Change'] = data['Price_Change%'].ewm(span=25, adjust=False).mean()  # Fast TSI
        data['Smooth_Price_Change_Slow'] = data['Price_Change%'].ewm(span=13, adjust=False).mean()  # Slow TSI
        data['TSI'] = 100 * (data['Smooth_Price_Change'] - data['Smooth_Price_Change_Slow']) / data['Smooth_Price_Change_Slow']
        data['Rolling_TSI'] = data['TSI'].rolling(window=rolling_window).mean()

        # ** 9. Rolling Volatility Calculation **
        data['Rolling_Volatility'] = data['Price_Change%'].rolling(window=rolling_window).std()

        # ** 10a. Rolling Maximum Consecutive Up Days Calculation **
        data['Consecutive_Up_Days'] = (data['Price_Change%'] > 0).astype(int)
        data['Rolling_Consecutive_Up_Days'] = data['Consecutive_Up_Days'].groupby((data['Consecutive_Up_Days'] != data['Consecutive_Up_Days'].shift()).cumsum()).cumsum()
        
        # ** 10b. Max Rolling Consecutive Up Days in a 20-day Window **
        consecutive_counts = data['Consecutive_Up_Days'].rolling(window=rolling_window).apply(
            lambda x: (x != 0).astype(int).groupby((x != 0).ne((x.shift())).cumsum()).cumsum().max(),
            raw=False
        )
        data['Max_Rolling_Consecutive_Up_Days'] = consecutive_counts

        # ** 11. RSI Calculation (Relative Strength Index) **
        delta = data['Close'].diff(1)
        gain = (delta.where(delta > 0, 0)).rolling(window=rolling_window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=rolling_window).mean()
        rs = gain / loss
        data['RSI'] = 100 - (100 / (1 + rs))

        # ** 12. Momentum Calculation **
        data['Momentum'] = data['Close'].diff(rolling_window)

        # ** 13. ATR Calculation (Average True Range) **
        data['High_Low'] = data['High'] - data['Low']
        data['High_Close'] = (data['High'] - data['Close'].shift(1)).abs()
        data['Low_Close'] = (data['Low'] - data['Close'].shift(1)).abs()
        data['True_Range'] = data[['High_Low', 'High_Close', 'Low_Close']].max(axis=1)
        data['ATR'] = data['True_Range'].rolling(window=rolling_window).mean()

        # ** 14. Beta Calculation **
        if 'benchmark_data' in locals():
            returns = data['Close'].pct_change()
            benchmark_returns = benchmark_data.pct_change()
            covariance = returns.rolling(window=rolling_window).cov(benchmark_returns)
            variance = benchmark_returns.rolling(window=rolling_window).var()

            data['Beta'] = covariance / variance

        # ** 15. Alpha Calculation **
        if 'benchmark_data' in locals():
            average_benchmark_return = benchmark_data.pct_change().rolling(window=rolling_window).mean()
            data['Alpha'] = (average_return - (data['Beta'] * average_benchmark_return)) * rolling_window

        # ** 16. Preparing the Trend Indicator DataFrame **
        trend_indicator = data[['Close', 'Price_Change%', 'Market_Trend', 'Cumulative_Change%', 
                                'Rolling_Drawdown', 'Rolling_Up_Down_Ratio', 'Rolling_TSI', 
                                'Rolling_Volatility', 'Rolling_Consecutive_Up_Days', 
                                'Max_Rolling_Consecutive_Up_Days', 'RSI', 'Momentum', 'ATR', 
                                'Beta', 'Alpha']].dropna()
        trend_indicator.columns = [f'{ticker} Close',
                                   f'{ticker} daily move%',
                                   f'{ticker} trend signal',
                                   f'{ticker} cumulative change since signal',
                                   f'{ticker} Rolling Max Drawdown',
                                   f'{ticker} Rolling Up Down Ratio',
                                   f'{ticker} Rolling TSI',
                                   f'{ticker} Rolling Volatility',
                                   f'{ticker} Consecutive Up Days',
                                   f'{ticker} Max Rolling Consecutive Up Days',
                                   f'{ticker} RSI',
                                   f'{ticker} Momentum',
                                   f'{ticker} ATR',
                                   f'{ticker} Beta',
                                   f'{ticker} Alpha']

        # ** 17. Sortino Ratio Calculation Section **
        average_return = data['Price_Change%'].rolling(window=rolling_window).mean()
        downside_returns = data['Price_Change%'][data['Price_Change%'] < target_return].rolling(window=rolling_window)
        downside_deviation = downside_returns.apply(lambda x: (x ** 2).mean() ** 0.5 if not x.empty else 0)

        # Calculate Sortino Ratio
        data['Sortino Ratio'] = (average_return - target_return) / downside_deviation
        data['Sortino Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities

        # ** 18. Calmar Ratio Calculation Section **
        annualized_return = ((1 + average_return / 100) ** 252) - 1
        data['Calmar Ratio'] = annualized_return / abs(data['Rolling_Drawdown'].min())  # Use current rolling drawdown

        # ** 19. Sharpe Ratio Calculation Section **
        trend_indicator[f'{ticker} Sortino Ratio'] = data['Sortino Ratio']
        trend_indicator[f'{ticker} Calmar Ratio'] = data['Calmar Ratio']

        results[ticker] = trend_indicator  # Store the results for this ticker

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# ** Risk-Free Rate Calculation Section **
if "^TNX" in results:  # Ensure ^TNX has been processed successfully
    treasury_yield = results["^TNX"]['^TNX Close']
    risk_free_rate = treasury_yield / 100 / 252  # Convert from annual to daily
else:
    print("Warning: ^TNX results not found. Defaulting risk-free rate to 0.")
    risk_free_rate = 0.0  # Fallback risk-free rate

# ** Calculate Sharpe Ratio for all tickers using the risk-free rate **
for ticker in tickers:
    if ticker in results:
        data = results[ticker]
        rolling_average_return = data[f'{ticker} daily move%'].rolling(window=rolling_window).mean()
        rolling_std_dev_returns = data[f'{ticker} daily move%'].rolling(window=rolling_window).std()

        # Define Sharpe Ratio; replace infinities for stability
        data[f'{ticker} Sharpe Ratio'] = (rolling_average_return - risk_free_rate) / rolling_std_dev_returns
        data[f'{ticker} Sharpe Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities
        results[ticker] = data  # Update results with Sharpe Ratio

# ** 20. Display Results Section **
for ticker in tickers:
    if ticker in results:  # Only display results for tickers that were processed successfully
        print(f"Results for {ticker}:")
        display(results[ticker].tail(10))  # Display the last 10 entries


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Error processing ^VIX: "['Beta', 'Alpha'] not in index"



[*********************100%%**********************]  1 of 1 completed

Error processing ^TNX: "['Beta', 'Alpha'] not in index"



[*********************100%%**********************]  1 of 1 completed

...

,GOVT Close,GOVT daily move%,GOVT trend signal,GOVT cumulative change since signal,GOVT Rolling Max Drawdown,GOVT Rolling Up Down Ratio,GOVT Rolling TSI,GOVT Rolling Volatility,GOVT Consecutive Up Days,GOVT Max Rolling Consecutive Up Days,GOVT RSI,GOVT Momentum,GOVT ATR,GOVT Beta,GOVT Alpha,GOVT Sortino Ratio,GOVT Calmar Ratio,GOVT Sharpe Ratio
Date,,,,,,,,,,,,,,,,,,
2024-10-23,22.860001,-0.174668,-1,-0.781251,-0.127814,0.298520,-27.058595,0.284085,0,2.0,22.935871,-0.590000,0.0880,-0.065025,-9.053692,-0.418508,-1.557394,-0.446852
2024-10-24,22.900000,0.174974,-1,-0.607644,-0.126288,0.351000,-22.195743,0.291554,1,2.0,25.892875,-0.540001,0.0885,-0.063168,-8.594909,NaN,-1.442494,-0.398085
2024-10-25,22.860001,-0.174668,-1,-0.781251,-0.127814,0.255050,-31.306271,0.274852,0,2.0,20.183560,-0.650000,0.0895,-0.049713,-9.834792,-0.460435,-1.688437,-0.508377
2024-10-28,22.830000,-0.131237,-1,-0.911463,-0.128958,0.263767,-26.450177,0.273505,0,2.0,20.754768,-0.620001,0.0895,-0.044043,-9.825341,-0.441373,-1.625424,-0.488217
2024-10-29,22.840000,0.043803,-1,-0.868059,-0.128577,0.275843,-21.060222,0.274803,1,2.0,21.495392,-0.610001,0.0875,-0.026728,-9.884041,NaN,-1.602924,-0.477941
2024-10-30,22.820000,-0.087568,-1,-0.954866,-0.129340,0.289264,-17.668985,0.273362,0,2.0,22.330182,-0.570000,0.0880,-0.030106,-7.499030,-0.422498,-1.515313,-0.449677
2024-10-31,22.820000,0.000000,-1,-0.954866,-0.129340,0.325468,-14.570960,0.267439,0,2.0,24.468176,-0.480000,0.0885,-0.047346,-5.966008,NaN,-1.307912,-0.387698
2024-11-01,22.670000,-0.657316,-1,-1.605906,-0.135063,0.333307,-13.242511,0.258885,0,2.0,25.000104,-0.459999,0.0875,-0.014620,-6.661523,-0.307128,-1.267811,-0.386545
2024-11-04,22.770000,0.441113,-1,-1.171877,-0.131248,0.534089,-10.461123,0.280750,1,2.0,34.736937,-0.289999,0.0905,-0.052684,-2.990224,NaN,-0.833433,-0.223982


Results for AAPL:


,AAPL Close,AAPL daily move%,AAPL trend signal,AAPL cumulative change since signal,AAPL Rolling Max Drawdown,AAPL Rolling Up Down Ratio,AAPL Rolling TSI,AAPL Rolling Volatility,AAPL Consecutive Up Days,AAPL Max Rolling Consecutive Up Days,AAPL RSI,AAPL Momentum,AAPL ATR,AAPL Beta,AAPL Alpha,AAPL Sortino Ratio,AAPL Calmar Ratio,AAPL Sharpe Ratio
Date,,,,,,,,,,,,,,,,,,
2024-10-23,230.759995,-2.162302,1,0.531498,-0.024188,1.214508,20.778625,1.403716,0,3.0,54.392636,4.389999,4.336499,1.605099,-2.561680,0.076111,1.000079,0.075145
2024-10-24,230.570007,-0.082331,-1,-0.082331,-0.024991,1.153198,12.772771,1.401010,0,3.0,53.111615,3.050003,4.302499,1.601481,-2.341124,0.055301,0.693070,0.054221
2024-10-25,231.410004,0.364313,-1,0.281682,-0.021439,1.177968,67.316203,1.402480,1,3.0,53.650678,3.620010,4.373999,1.606506,-2.816263,NaN,0.818062,0.062922
2024-10-28,233.399994,0.859941,-1,1.144046,-0.013024,1.034049,69.351280,1.318498,2,3.0,50.431400,0.399994,4.279498,1.537577,-2.689952,NaN,0.142828,0.012805
2024-10-29,233.669998,0.115683,1,0.115683,-0.011883,1.480873,22.566690,1.123671,3,3.0,59.362447,7.459991,3.916998,1.283801,-2.656533,NaN,1.735165,0.149844
2024-10-30,230.100006,-1.527792,1,-1.413877,-0.026979,1.186121,13.153267,1.185476,0,3.0,53.874895,3.320007,3.905499,1.363216,-2.485371,0.056406,0.727522,0.066967
2024-10-31,225.910004,-1.820948,-1,-1.820948,-0.044697,1.025984,19.747082,1.254481,0,3.0,50.261330,0.240005,3.967500,1.180773,-2.076005,0.008757,0.107837,0.010214
2024-11-01,222.910004,-1.327962,-1,-3.124729,-0.057383,0.859480,13.865114,1.283358,0,3.0,45.930108,-3.889999,4.056000,1.152012,-1.998045,-0.057667,-0.590888,-0.061263
2024-11-04,222.009995,-0.403754,-1,-3.515868,-0.061189,1.029643,14.455711,1.180980,0,3.0,50.367132,0.319992,3.942500,1.030467,-1.261747,0.010235,0.116673,0.011723


Results for SFM:


,SFM Close,SFM daily move%,SFM trend signal,SFM cumulative change since signal,SFM Rolling Max Drawdown,SFM Rolling Up Down Ratio,SFM Rolling TSI,SFM Rolling Volatility,SFM Consecutive Up Days,SFM Max Rolling Consecutive Up Days,SFM RSI,SFM Momentum,SFM ATR,SFM Beta,SFM Alpha,SFM Sortino Ratio,SFM Calmar Ratio,SFM Sharpe Ratio
Date,,,,,,,,,,,,,,,,,,
2024-10-23,118.849998,0.490405,1,1.217851,0.000000,2.327346,4.790009,1.191591,3,4.0,69.840540,8.709999,2.295500,0.126018,2.107841,NaN,4.881198,0.325592
2024-10-24,118.779999,-0.058898,1,1.158236,-0.000589,3.100225,4.936142,1.110840,0,4.0,75.331682,10.309998,2.183000,0.210276,1.516670,0.338994,6.452895,0.414858
2024-10-25,119.320000,0.454623,1,1.618124,0.000000,3.648350,3.720065,1.085922,1,4.0,78.138841,11.430000,2.197500,0.180463,1.762487,NaN,7.695119,0.469930
2024-10-28,120.180000,0.720751,1,2.350538,0.000000,3.229289,4.253281,0.999660,2,4.0,76.193018,9.769997,2.113000,0.098608,0.336427,NaN,5.742987,0.429705
2024-10-29,119.690002,-0.407720,1,1.933235,-0.004077,2.801307,5.616966,1.016708,0,4.0,73.466094,8.720001,2.092001,0.121854,3.364689,0.281706,4.799823,0.377505
2024-10-30,118.839996,-0.710173,1,1.209332,-0.011150,2.529534,9.114619,1.036743,0,4.0,71.279347,8.149994,2.131001,0.185892,1.584079,0.265211,4.366821,0.348129
2024-10-31,128.429993,8.069671,1,9.376592,0.000000,4.172447,7.314063,2.011008,1,3.0,80.626959,17.389992,2.863001,-1.410746,0.259001,NaN,16.385267,0.372248
2024-11-01,130.210007,1.385980,1,10.892530,0.000000,4.111190,6.386353,2.005028,2,3.0,80.572223,17.310005,2.925502,-1.570846,-1.577028,NaN,15.698552,0.366149
2024-11-04,131.789993,1.213414,1,12.238116,0.000000,4.336399,5.320638,2.002831,3,3.0,81.493923,18.719994,3.055002,-1.789168,0.283987,NaN,18.349668,0.393084


In [3]:
import yfinance as yf
import pandas as pd

# List of tickers to analyze including the benchmark
tickers = ['^VIX', '^TNX', '^IRX',
           'VXX', 'UVXY',
           'MTUM', 
           'SPY', 'QQQ', 'QQQM', 'VOOG', 'VGT', 'XLU',
           'GLD', 'IAU', 'PHYS',
           'TLT', 'TMF', 'SHY', 'EDV', 'ZROZ', 'GOVT',
           'AAPL', 'SFM']

# Initialize a dictionary to store results for each ticker
results = {}

# Define a target return (for instance, the risk-free rate)
target_return = 0.0  # Adjust this as needed
rolling_window = 20   # Define the window size for rolling calculations

# Define the benchmark
benchmark_ticker = 'SPY'

# ** Data Processing Loop for Each Ticker **
for ticker in tickers:
    # ** 1. Data Downloading Section **
    try:
        data = yf.download(ticker, start="2022-01-01", end="2024-12-31", interval="1d")

        # Check if the data is empty
        if data.empty:
            print(f"No data found for {ticker}. Skipping.")
            continue
        
        # Add benchmark data for Beta calculation
        if ticker == benchmark_ticker:
            benchmark_data = data['Close']
            continue  # Skip the rest for the benchmark itself

        # ** 2. Moving Averages Section **
        data['3_SMA'] = data['Close'].rolling(window=3).mean()
        data['5_SMA'] = data['Close'].rolling(window=5).mean()
        data['20_SMA'] = data['Close'].rolling(window=20).mean()
        data['50_SMA'] = data['Close'].rolling(window=50).mean()

        # ** 3. Market Trend Section **
        data['Market_Trend'] = 0
        data.loc[data['3_SMA'] > data['5_SMA'], 'Market_Trend'] = 1
        data.loc[data['3_SMA'] < data['5_SMA'], 'Market_Trend'] = -1

        # ** 4. Price Change Calculations Section **
        data['Price_Change%'] = data['Close'].pct_change() * 100 
        data['Cumulative_Returns'] = (1 + data['Price_Change%'] / 100).cumprod()

        # ** 5. Rolling Maximum Drawdown Calculation Section **
        data['Rolling_Peak'] = data['Cumulative_Returns'].cummax()
        data['Rolling_Drawdown'] = (data['Cumulative_Returns'] - data['Rolling_Peak']) / data['Rolling_Peak']

        # ** 6. Cumulative Change Calculation Section using vectorized operations **
        current_trend_shifted = data['Market_Trend'].shift(1)
        current_base_price = data['Close'].shift(1).where(current_trend_shifted != data['Market_Trend']).ffill()
        data['Cumulative_Change%'] = ((data['Close'] - current_base_price) / current_base_price) * 100

        # ** 7. Rolling Up/Down Ratio Calculation **
        data['Up_Day'] = data['Price_Change%'].apply(lambda x: x if x > 0 else 0)
        data['Down_Day'] = data['Price_Change%'].apply(lambda x: -x if x < 0 else 0)
        data['Rolling_Up_Down_Ratio'] = data['Up_Day'].rolling(window=rolling_window).mean() / data['Down_Day'].rolling(window=rolling_window).mean()
        
        # ** 8. Rolling True Strength Index (TSI) Calculation **
        data['Smooth_Price_Change'] = data['Price_Change%'].ewm(span=25, adjust=False).mean()  # Fast TSI
        data['Smooth_Price_Change_Slow'] = data['Price_Change%'].ewm(span=13, adjust=False).mean()  # Slow TSI
        data['TSI'] = 100 * (data['Smooth_Price_Change'] - data['Smooth_Price_Change_Slow']) / data['Smooth_Price_Change_Slow']
        data['Rolling_TSI'] = data['TSI'].rolling(window=rolling_window).mean()

        # ** 9. Rolling Volatility Calculation **
        data['Rolling_Volatility'] = data['Price_Change%'].rolling(window=rolling_window).std()

        # ** 10a. Rolling Maximum Consecutive Up Days Calculation **
        data['Consecutive_Up_Days'] = (data['Price_Change%'] > 0).astype(int)
        data['Rolling_Consecutive_Up_Days'] = data['Consecutive_Up_Days'].groupby((data['Consecutive_Up_Days'] != data['Consecutive_Up_Days'].shift()).cumsum()).cumsum()
        
        # ** 10b. Max Rolling Consecutive Up Days in a 20-day Window **
        consecutive_counts = data['Consecutive_Up_Days'].rolling(window=rolling_window).apply(
            lambda x: (x != 0).astype(int).groupby((x != 0).ne((x.shift())).cumsum()).cumsum().max(),
            raw=False
        )
        data['Max_Rolling_Consecutive_Up_Days'] = consecutive_counts

        # ** 11. RSI Calculation (Relative Strength Index) **
        delta = data['Close'].diff(1)
        gain = (delta.where(delta > 0, 0)).rolling(window=rolling_window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=rolling_window).mean()
        rs = gain / loss
        data['RSI'] = 100 - (100 / (1 + rs))

        # ** 12. Momentum Calculation **
        data['Momentum'] = data['Close'].diff(rolling_window)

        # ** 13. ATR Calculation (Average True Range) **
        data['High_Low'] = data['High'] - data['Low']
        data['High_Close'] = (data['High'] - data['Close'].shift(1)).abs()
        data['Low_Close'] = (data['Low'] - data['Close'].shift(1)).abs()
        data['True_Range'] = data[['High_Low', 'High_Close', 'Low_Close']].max(axis=1)
        data['ATR'] = data['True_Range'].rolling(window=rolling_window).mean()

        # ** 14. Beta Calculation **
        if 'benchmark_data' in locals():
            returns = data['Close'].pct_change()
            benchmark_returns = benchmark_data.pct_change()
            covariance = returns.rolling(window=rolling_window).cov(benchmark_returns)
            variance = benchmark_returns.rolling(window=rolling_window).var()

            data['Beta'] = covariance / variance

        # ** 15. Alpha Calculation **
        if 'benchmark_data' in locals():
            average_benchmark_return = benchmark_data.pct_change().rolling(window=rolling_window).mean()
            data['Alpha'] = (data['Price_Change%'].rolling(window=rolling_window).mean() - (data['Beta'] * average_benchmark_return)).fillna(0)

        # ** 16. Preparing the Trend Indicator DataFrame **
        trend_indicator = data[['Close', 'Price_Change%', 'Market_Trend', 'Cumulative_Change%', 
                                'Rolling_Drawdown', 'Rolling_Up_Down_Ratio', 'Rolling_TSI', 
                                'Rolling_Volatility', 'Rolling_Consecutive_Up_Days', 
                                'Max_Rolling_Consecutive_Up_Days', 'RSI', 'Momentum', 'ATR']].dropna()

        # Conditionally adding Alpha and Beta if they exist in the DataFrame
        trend_columns = [
            f'{ticker} Close',
            f'{ticker} daily move%',
            f'{ticker} trend signal',
            f'{ticker} cumulative change since signal',
            f'{ticker} Rolling Max Drawdown',
            f'{ticker} Rolling Up Down Ratio',
            f'{ticker} Rolling TSI',
            f'{ticker} Rolling Volatility',
            f'{ticker} Consecutive Up Days',
            f'{ticker} Max Rolling Consecutive Up Days',
            f'{ticker} RSI',
            f'{ticker} Momentum',
            f'{ticker} ATR'
        ]

        # Check if Alpha and Beta exist in data for column addition
        if 'Alpha' in data.columns:
            trend_indicator[f'{ticker} Alpha'] = data['Alpha']
            trend_columns.append(f'{ticker} Alpha')

        if 'Beta' in data.columns:
            trend_indicator[f'{ticker} Beta'] = data['Beta']
            trend_columns.append(f'{ticker} Beta')

        trend_indicator.columns = trend_columns

        # ** 17. Sortino Ratio Calculation Section **
        average_return = data['Price_Change%'].rolling(window=rolling_window).mean()
        downside_returns = data['Price_Change%'][data['Price_Change%'] < target_return].rolling(window=rolling_window)
        downside_deviation = downside_returns.apply(lambda x: (x ** 2).mean() ** 0.5 if not x.empty else 0)

        # Calculate Sortino Ratio
        data['Sortino Ratio'] = (average_return - target_return) / downside_deviation
        data['Sortino Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities

        # ** 18. Calmar Ratio Calculation Section **
        annualized_return = ((1 + average_return / 100) ** 252) - 1
        data['Calmar Ratio'] = annualized_return / abs(data['Rolling_Drawdown'].min())  # Use current rolling drawdown

        # ** 19. Sharpe Ratio Calculation Section **
        trend_indicator[f'{ticker} Sortino Ratio'] = data['Sortino Ratio']
        trend_indicator[f'{ticker} Calmar Ratio'] = data['Calmar Ratio']

        results[ticker] = trend_indicator  # Store the results for this ticker

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# ** Risk-Free Rate Calculation Section **
if "^TNX" in results:  # Ensure ^TNX has been processed successfully
    treasury_yield = results["^TNX"]['^TNX Close']
    risk_free_rate = treasury_yield / 100 / 252  # Convert from annual to daily
else:
    print("Warning: ^TNX results not found. Defaulting risk-free rate to 0.")
    risk_free_rate = 0.0  # Fallback risk-free rate

# ** Calculate Sharpe Ratio for all tickers using the risk-free rate **
for ticker in tickers:
    if ticker in results:
        data = results[ticker]
        rolling_average_return = data[f'{ticker} daily move%'].rolling(window=rolling_window).mean()
        rolling_std_dev_returns = data[f'{ticker} daily move%'].rolling(window=rolling_window).std()

        # Define Sharpe Ratio; replace infinities for stability
        data[f'{ticker} Sharpe Ratio'] = (rolling_average_return - risk_free_rate) / rolling_std_dev_returns
        data[f'{ticker} Sharpe Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)  # Replace infinities
        results[ticker] = data  # Update results with Sharpe Ratio

# ** 20. Display Results Section **
for ticker in tickers:
    if ticker in results:  # Only display results for tickers that were processed successfully
        print(f"Results for {ticker}:")
        display(results[ticker].tail(10))  # Display the last 10 entries


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Results for ^VIX:


,^VIX Close,^VIX daily move%,^VIX trend signal,^VIX cumulative change since signal,^VIX Rolling Max Drawdown,^VIX Rolling Up Down Ratio,^VIX Rolling TSI,^VIX Rolling Volatility,^VIX Consecutive Up Days,^VIX Max Rolling Consecutive Up Days,^VIX RSI,^VIX Momentum,^VIX ATR,^VIX Sortino Ratio,^VIX Calmar Ratio,^VIX Sharpe Ratio
Date,,,,,,,,,,,,,,,,
2024-10-24,19.080000,-0.831600,1,4.835160,-0.505315,1.679064,-1.171510,6.940018,0,1.0,59.586566,3.710000,1.7310,0.306563,37.234029,0.187746
2024-10-25,20.330000,6.551363,1,11.703292,-0.472906,1.580224,-0.502782,6.728515,1,1.0,58.863760,3.370001,1.7565,NaN,22.660631,0.165457
2024-10-28,19.799999,-2.606988,1,8.791200,-0.486648,1.530348,0.851402,6.758425,0,1.0,57.949249,3.070000,1.7615,0.264761,19.175040,0.155471
2024-10-29,19.340000,-2.323228,1,6.263733,-0.498574,1.085150,12.974732,5.920554,0,1.0,50.232018,0.080000,1.6290,0.045501,0.841765,0.030141
2024-10-30,20.350000,5.222338,1,11.813184,-0.472388,1.266032,13.922333,6.003234,1,1.0,54.052546,1.450001,1.5970,NaN,4.179157,0.088790
2024-10-31,23.160000,13.808351,1,27.252741,-0.399533,1.400638,13.990984,6.478405,2,2.0,56.985873,2.670000,1.6580,NaN,9.640785,0.123921
2024-11-01,21.879999,-5.526773,1,20.219771,-0.432720,1.426263,14.053287,6.439039,0,2.0,56.985873,2.670000,1.6575,0.206596,10.688340,0.130270
2024-11-04,21.980000,0.457040,1,20.769223,-0.430127,0.984287,14.906675,5.043076,1,2.0,47.908745,-0.660000,1.5335,NaN,-0.111151,-0.006166
2024-11-05,20.490000,-6.778889,-1,-6.778889,-0.468758,0.950712,23.334868,5.129648,0,2.0,47.102802,-0.930000,1.5265,-0.027480,-0.331604,-0.019613


Results for ^TNX:


,^TNX Close,^TNX daily move%,^TNX trend signal,^TNX cumulative change since signal,^TNX Rolling Max Drawdown,^TNX Rolling Up Down Ratio,^TNX Rolling TSI,^TNX Rolling Volatility,^TNX Consecutive Up Days,^TNX Max Rolling Consecutive Up Days,^TNX RSI,^TNX Momentum,^TNX ATR,^TNX Sortino Ratio,^TNX Calmar Ratio,^TNX Sharpe Ratio
Date,,,,,,,,,,,,,,,,
2024-10-23,4.242,0.903904,1,4.149279,-0.149559,3.020263,-17.941657,1.314763,3,7.0,75.081585,0.461,0.06390,NaN,12.220964,0.444849
2024-10-24,4.200,-0.990106,1,3.118091,-0.157979,2.540324,-14.480620,1.360013,0,7.0,71.503639,0.409,0.06360,0.477058,9.912329,0.383925
2024-10-25,4.232,0.761906,1,3.903755,-0.151564,3.170624,-7.265197,1.305215,1,7.0,75.664125,0.483,0.06390,NaN,13.492125,0.471671
2024-10-28,4.278,1.086957,1,5.033143,-0.142342,3.113036,-4.669204,1.296719,2,7.0,75.481737,0.476,0.06455,NaN,12.804836,0.462160
2024-10-29,4.274,-0.093495,1,4.934943,-0.143143,4.189937,-23.262449,1.207302,0,7.0,80.204730,0.531,0.06260,0.635660,16.100823,0.556785
2024-10-30,4.266,-0.187187,1,4.738518,-0.144747,3.756941,-19.618768,1.217091,0,6.0,78.461456,0.481,0.06290,0.587898,13.114847,0.498520
2024-10-31,4.284,0.421944,1,5.180456,-0.141139,3.462729,-16.844282,1.189034,1,5.0,77.192907,0.434,0.06330,NaN,10.603659,0.455812
2024-11-01,4.361,1.797389,1,7.070958,-0.125702,3.098141,-15.206909,1.029227,2,4.0,75.537562,0.380,0.06385,NaN,8.005582,0.448601
2024-11-04,4.309,-1.192388,1,5.794256,-0.136127,2.235873,-12.438354,1.079641,0,3.0,68.841486,0.283,0.06615,0.339783,5.058743,0.320085


...

,GOVT Close,GOVT daily move%,GOVT trend signal,GOVT cumulative change since signal,GOVT Rolling Max Drawdown,GOVT Rolling Up Down Ratio,GOVT Rolling TSI,GOVT Rolling Volatility,GOVT Consecutive Up Days,GOVT Max Rolling Consecutive Up Days,GOVT RSI,GOVT Momentum,GOVT ATR,GOVT Alpha,GOVT Beta,GOVT Sortino Ratio,GOVT Calmar Ratio,GOVT Sharpe Ratio
Date,,,,,,,,,,,,,,,,,,
2024-10-23,22.860001,-0.174668,-1,-0.781251,-0.127814,0.298520,-27.058595,0.284085,0,2.0,22.935871,-0.590000,0.0880,-0.126898,-0.065025,-0.418508,-1.557394,-0.447444
2024-10-24,22.900000,0.174974,-1,-0.607644,-0.126288,0.351000,-22.195743,0.291554,1,2.0,25.892875,-0.540001,0.0885,-0.116024,-0.063168,NaN,-1.442494,-0.398657
2024-10-25,22.860001,-0.174668,-1,-0.781251,-0.127814,0.255050,-31.306271,0.274852,0,2.0,20.183560,-0.650000,0.0895,-0.139695,-0.049713,-0.460435,-1.688437,-0.508988
2024-10-28,22.830000,-0.131237,-1,-0.911463,-0.128958,0.263767,-26.450177,0.273505,0,2.0,20.754768,-0.620001,0.0895,-0.133502,-0.044043,-0.441373,-1.625424,-0.488837
2024-10-29,22.840000,0.043803,-1,-0.868059,-0.128577,0.275843,-21.060222,0.274803,1,2.0,21.495392,-0.610001,0.0875,-0.131309,-0.026728,NaN,-1.602924,-0.478558
2024-10-30,22.820000,-0.087568,-1,-0.954866,-0.129340,0.289264,-17.668985,0.273362,0,2.0,22.330182,-0.570000,0.0880,-0.122895,-0.030106,-0.422498,-1.515313,-0.450296
2024-10-31,22.820000,0.000000,-1,-0.954866,-0.129340,0.325468,-14.570960,0.267439,0,2.0,24.468176,-0.480000,0.0885,-0.103681,-0.047346,NaN,-1.307912,-0.388333
2024-11-01,22.670000,-0.657316,-1,-1.605906,-0.135063,0.333307,-13.242511,0.258885,0,2.0,25.000104,-0.459999,0.0875,-0.100073,-0.014620,-0.307128,-1.267811,-0.387213
2024-11-04,22.770000,0.441113,-1,-1.171877,-0.131248,0.534089,-10.461123,0.280750,1,2.0,34.736937,-0.289999,0.0905,-0.062873,-0.052684,NaN,-0.833433,-0.224591


Results for AAPL:


,AAPL Close,AAPL daily move%,AAPL trend signal,AAPL cumulative change since signal,AAPL Rolling Max Drawdown,AAPL Rolling Up Down Ratio,AAPL Rolling TSI,AAPL Rolling Volatility,AAPL Consecutive Up Days,AAPL Max Rolling Consecutive Up Days,AAPL RSI,AAPL Momentum,AAPL ATR,AAPL Alpha,AAPL Beta,AAPL Sortino Ratio,AAPL Calmar Ratio,AAPL Sharpe Ratio
Date,,,,,,,,,,,,,,,,,,
2024-10-23,230.759995,-2.162302,1,0.531498,-0.024188,1.214508,20.778625,1.403716,0,3.0,54.392636,4.389999,4.336499,0.104342,1.605099,0.076111,1.000079,0.075025
2024-10-24,230.570007,-0.082331,-1,-0.082331,-0.024991,1.153198,12.772771,1.401010,0,3.0,53.111615,3.050003,4.302499,0.074971,1.601481,0.055301,0.693070,0.054102
2024-10-25,231.410004,0.364313,-1,0.281682,-0.021439,1.177968,67.316203,1.402480,1,3.0,53.650678,3.620010,4.373999,0.087161,1.606506,NaN,0.818062,0.062802
2024-10-28,233.399994,0.859941,-1,1.144046,-0.013024,1.034049,69.351280,1.318498,2,3.0,50.431400,0.399994,4.279498,0.015916,1.537577,NaN,0.142828,0.012676
2024-10-29,233.669998,0.115683,1,0.115683,-0.011883,1.480873,22.566690,1.123671,3,3.0,59.362447,7.459991,3.916998,0.166889,1.283801,NaN,1.735165,0.149693
2024-10-30,230.100006,-1.527792,1,-1.413877,-0.026979,1.186121,13.153267,1.185476,0,3.0,53.874895,3.320007,3.905499,0.078043,1.363216,0.056406,0.727522,0.066824
2024-10-31,225.910004,-1.820948,-1,-1.820948,-0.044697,1.025984,19.747082,1.254481,0,3.0,50.261330,0.240005,3.967500,0.012698,1.180773,0.008757,0.107837,0.010078
2024-11-01,222.910004,-1.327962,-1,-3.124729,-0.057383,0.859480,13.865114,1.283358,0,3.0,45.930108,-3.889999,4.056000,-0.078454,1.152012,-0.057667,-0.590888,-0.061397
2024-11-04,222.009995,-0.403754,-1,-3.515868,-0.061189,1.029643,14.455711,1.180980,0,3.0,50.367132,0.319992,3.942500,0.013640,1.030467,0.010235,0.116673,0.011578


Results for SFM:


,SFM Close,SFM daily move%,SFM trend signal,SFM cumulative change since signal,SFM Rolling Max Drawdown,SFM Rolling Up Down Ratio,SFM Rolling TSI,SFM Rolling Volatility,SFM Consecutive Up Days,SFM Max Rolling Consecutive Up Days,SFM RSI,SFM Momentum,SFM ATR,SFM Alpha,SFM Beta,SFM Sortino Ratio,SFM Calmar Ratio,SFM Sharpe Ratio
Date,,,,,,,,,,,,,,,,,,
2024-10-23,118.849998,0.490405,1,1.217851,0.000000,2.327346,4.790009,1.191591,3,4.0,69.840540,8.709999,2.295500,0.387883,0.126018,NaN,4.881198,0.325451
2024-10-24,118.779999,-0.058898,1,1.158236,-0.000589,3.100225,4.936142,1.110840,0,4.0,75.331682,10.309998,2.183000,0.460710,0.210276,0.338994,6.452895,0.414707
2024-10-25,119.320000,0.454623,1,1.618124,0.000000,3.648350,3.720065,1.085922,1,4.0,78.138841,11.430000,2.197500,0.510185,0.180463,NaN,7.695119,0.469775
2024-10-28,120.180000,0.720751,1,2.350538,0.000000,3.229289,4.253281,0.999660,2,4.0,76.193018,9.769997,2.113000,0.429497,0.098608,NaN,5.742987,0.429535
2024-10-29,119.690002,-0.407720,1,1.933235,-0.004077,2.801307,5.616966,1.016708,0,4.0,73.466094,8.720001,2.092001,0.383672,0.121854,0.281706,4.799823,0.377338
2024-10-30,118.839996,-0.710173,1,1.209332,-0.011150,2.529534,9.114619,1.036743,0,4.0,71.279347,8.149994,2.131001,0.360737,0.185892,0.265211,4.366821,0.347966
2024-10-31,128.429993,8.069671,1,9.376592,0.000000,4.172447,7.314063,2.011008,1,3.0,80.626959,17.389992,2.863001,0.748731,-1.410746,NaN,16.385267,0.372164
2024-11-01,130.210007,1.385980,1,10.892530,0.000000,4.111190,6.386353,2.005028,2,3.0,80.572223,17.310005,2.925502,0.733910,-1.570846,NaN,15.698552,0.366063
2024-11-04,131.789993,1.213414,1,12.238116,0.000000,4.336399,5.320638,2.002831,3,3.0,81.493923,18.719994,3.055002,0.787636,-1.789168,NaN,18.349668,0.392999
